In [9]:
# -*- coding: utf-8 -*-
import pymysql
from pymysql.cursors import DictCursor


connection = pymysql.connect(
            host='localhost',
            user='root',
            password='',
            db='egais',
            charset='utf8mb4',
            cursorclass=DictCursor)


cursor=connection.cursor()
result=[]
cursor.execute('SELECT* FROM kassa_kassir ')
for row1 in cursor: result.append(row1)
spisok_prodavc=[]
for i in result:
    spisok_prodavc.append(i['fio'])
    

cursor=connection.cursor()
organizac=[]
ul=[]
cursor.execute("SELECT ul_id FROM kassa_ulonsaleplace")
for row1 in cursor: ul.append(row1)

for i in range (0,len(ul)):
    cursor.execute('SELECT shortname, address, fsrar FROM kassa_ul where id ='+str(ul[i].get('ul_id')))
    for row1 in cursor: organizac.append(row1)
spisok_origanizac=[]
for i in range(0,len(organizac)):
    spisok_origanizac.append(organizac[i].get('shortname'))


cursor=connection.cursor()
kategor=[]
cursor.execute("SELECT name FROM kassa_categ")
for row1 in cursor: kategor.append(row1)
for i in range(0,len(kategor)):
    kategor[i]=kategor[i].get("name")
kategor=kategor[2:len(kategor)-1]

from itertools import groupby
from collections import Counter
import sip
import urllib.request
import xml.etree.ElementTree as xml
import sys 
from PyQt5 import QtWidgets
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
import threading
#from PyQt5.QtWidgets import QApplication, QMainWindow, QGridLayout, QWidget, QTableWidget, QTableWidgetItem,QCheckBox
import sys
import time
import datetime
from PyQt5.QtGui import QPixmap
from PyQt5 import QtCore, QtGui, QtWidgets,QtCore
from PyQt5.QtWidgets import QApplication, QWidget,QVBoxLayout,  QToolBox,QPushButton, QGridLayout,QTableWidgetItem, QLineEdit,QInputDialog,QApplication,QLabel, QTableWidget, QTableWidgetItem, QHBoxLayout,QComboBox,QLineEdit
from PyQt5.QtCore import QDate, QTime, QDateTime, Qt
from time import strftime
from PyQt5.QtCore import QTime, QTimer
from PyQt5.QtWidgets import QApplication, QLCDNumber
from PyQt5.QtCore import QThread, pyqtSignal
import sip
from PyQt5.QtCore import QBasicTimer
LastStateRole = QtCore.Qt.UserRole

import design_new  # Это наш конвертированный файл дизайна
import design_new_dop_okno
from design_new_dop_okno import Ui_Dialog as Form
import design_okno_opl
import gurnal_pr

class gurnal_pr(QtWidgets.QMainWindow, gurnal_pr.Ui_MainWindow):

    def __init__(self):
        super().__init__()
        # Это здесь нужно для доступа к переменным, методам
        self.setupUi(self)
        self.initUi()

    def initUi(self):
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2
        self.comboBox.addItems(spisok_origanizac) 
        self.label.setText("c")
        self.dateEdit.setDate(datetime.datetime.now().date())
        self.dateEdit_2.setDate(datetime.datetime.now().date())
        self.label_2.setText("по")
        self.label_3.setText("номер смены")
        self.table = self.tableWidget
        self.table.setColumnCount(6)
        self.table.setGridStyle(0)
        self.table.setHorizontalHeaderLabels(["№ смены","Дата/Время", "Продукция","Сумма чека","ФИО кассира","Оплата"])
        fnt = self.table.font()
        fnt.setPointSize(30)

        for i in range(0,6):
            self.table.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)

        self.table.setSelectionBehavior(QAbstractItemView.SelectRows)
        #колонки по размерности текста
        header = self.table.horizontalHeader()       
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)
        header.setSectionResizeMode(1, QtWidgets.QHeaderView.ResizeToContents)
        header.setSectionResizeMode(2, QtWidgets.QHeaderView.ResizeToContents)
        #колонка названия товара растягивается максимально
        header.setSectionResizeMode(3, QtWidgets.QHeaderView.ResizeToContents)
        for i in range(4,6):
            header.setSectionResizeMode(i, QtWidgets.QHeaderView.ResizeToContents)

        indices = int(self.table.selectionModel().currentIndex().row())
        
        self.pushButton.clicked.connect(self.zapoln_tabl)

        self.zapoln_tabl()
        #time = datetime.datetime.now().time()
        #value=times[0].get("begintime")
        #time_morning=(datetime.datetime.min + value).time()

        
        
        
    def zapoln_tabl(self):
        self.table.clear()
        self.table.setRowCount( 0)
        self.table.setColumnCount(6)
        self.table.setGridStyle(0)
        
        self.table.setHorizontalHeaderLabels(["№ смены","Дата/Время", "Продукция","Сумма чека","ФИО кассира","Оплата"])
        a=self.dateEdit.text().replace('.','-')
        
        print(a[6:]+a[2:6]+a[0:2]+" 00:00:00")
        b=self.dateEdit_2.text().replace('.','-')
        id_ul=[]
        cursor=connection.cursor() 
        cursor.execute("SELECT id FROM kassa_ul where shortname ='"+self.comboBox.currentText()+"'")
        for row1 in cursor:id_ul.append(row1)
        print(id_ul)
        kassa_check=[]
        if self.lineEdit.text()=='':
            cursor2=connection.cursor()   
            cursor2.execute("""SELECT ID,DT_CLOSE,SMENA,KASSIR_FIO,MODE FROM KASSA_CHECK WHERE DT_CLOSE>='"""+
            a[6:]+a[2:6]+a[0:2]+" 00:00:00'"+" AND DT_CLOSE<='"""+b[6:]+b[2:6]+b[0:2]+"23:59:59'"+"AND kassa_ulonsaleplace_id="+str(id_ul[0].get("id")))
            for row1 in cursor2:kassa_check.append(row1)
            print(self.comboBox.currentText())
        else:
            cursor2=connection.cursor()   
            cursor2.execute("""SELECT ID,DT_CLOSE,SMENA,KASSIR_FIO,MODE FROM KASSA_CHECK WHERE SMENA="""+self.lineEdit.text()+""" AND DT_CLOSE>='"""+
            a[6:]+a[2:6]+a[0:2]+" 00:00:00'"+" AND DT_CLOSE<='"""+b[6:]+b[2:6]+b[0:2]+"23:59:59'"+"AND kassa_ulonsaleplace_id="+str(id_ul[0].get("id")))
            for row1 in cursor2:kassa_check.append(row1)            
        
        kassa_check_positions=[]
        price=[]
        for i in kassa_check:
            cursor2=connection.cursor() 
            cursor2.execute("""SELECT KASSA_CHECK_ID,KASSA_GOODS_ID,MARKA,FULLNAME,PRICE FROM kassa_check_positions WHERE KASSA_CHECK_ID="""+str(i.get("ID")))
            for row1 in cursor2:kassa_check_positions.append(row1)
            cursor2=connection.cursor() 
            cursor2.execute("""SELECT SUM(PRICE)  FROM kassa_check_positions WHERE KASSA_CHECK_ID="""+str(i.get("ID"))+" GROUP BY KASSA_CHECK_ID")
            for row1 in cursor2:price.append(row1)
        
        #print(kassa_check_positions) 
        spisok_fullname=[]
        full=[]
        param=[]
        A=[]
        B=[]
        for i in kassa_check:
            cursor2=connection.cursor() 
            cursor2.execute("""SELECT kassa_check_id,FULLNAME,price, shtrih, marka,param FROM kassa_check_positions WHERE KASSA_CHECK_ID="""+str(i.get("ID")))
            for i in cursor2: full.append(i)
        connection.commit()        
        for i in kassa_check:       
            
            cursor=connection.cursor()
            cursor.execute("""SELECT param FROM kassa_check_positions WHERE KASSA_CHECK_ID="""+str(i.get("ID")))            
            for i in cursor: param.append(i)

        for i in range(0,len(full)):
            full[i]=full[i].values()
        for i in range(0,len(full)):
            full[i]=list(full[i])
        a=[]
        c=[]

        for i in range(0,len(full)):
            a.append(str(full[i][1]))
            a.append("Объём: "+param[i].get("param")[13:param[i].get("param").find(",")-1])
            a.append("Цена: "+str(full[i][2])+" Руб.")
            a.append('\n')
            a.append("Штрих-код: "+full[i][3])
            a.append('\n')
            if full[i][4]!='':
                a.append("Марка: "+full[i][4])
            else: a.append(" ")
            a.append('\n')
            a.append('                    ')
            b='  '.join(a)
            c.append(full[i][0])
            c.append(b)
            full[i]=c
            a=[]
            c=[]

            
            
            
            
        sorted_vehicles=full
        A=[] 
        for key, group in groupby(sorted_vehicles, lambda make: make[0]):
            for make, model in group:
                
                A.append(model)
                #print('{model} is made by {make}'.format(model=model, make=make))

            B.append(A)
            A=[]
        for i in range(0,len(B)):

            B[i]= '\n'.join(B[i])   
                
        
        
            
        for i in range(0,len(price)):
        
            self.table.insertRow(i) 

            self.table.setRowHeight(i,60) 


            self.table.setItem(i, 0, QTableWidgetItem(str(kassa_check[i].get('SMENA'))))     
            self.table.setItem(i, 1, QTableWidgetItem(str(kassa_check[i].get('DT_CLOSE'))) ) 
            self.table.setItem(i, 2, QTableWidgetItem(B[i] ) )
            self.table.setItem(i, 3, QTableWidgetItem(str(price[i].get('SUM(PRICE)')) +" Руб.") )
            self.table.setItem(i, 4, QTableWidgetItem(str(kassa_check[i].get('KASSIR_FIO')))) 
            if kassa_check[i].get('MODE')==1:
                self.table.setItem(i, 5, QTableWidgetItem("карта") )
            else:
                self.table.setItem(i, 5, QTableWidgetItem("наличные") )
                
            
            if i%2!=0:
                self.table.item(i, 0).setBackground(QtGui.QColor(255, 255, 255))
                self.table.item(i, 1).setBackground(QtGui.QColor(255, 255, 255))
                self.table.item(i, 2).setBackground(QtGui.QColor(255, 255, 255))
                self.table.item(i, 3).setBackground(QtGui.QColor(255, 255, 255))
                self.table.item(i, 4).setBackground(QtGui.QColor(255, 255, 255))
                self.table.item(i, 5).setBackground(QtGui.QColor(255, 255, 255))
            self.table.item(i,0).setFlags(QtCore.Qt.ItemIsEnabled )
            self.table.item(i,1).setFlags(QtCore.Qt.ItemIsEnabled)
            self.table.item(i,2).setFlags(QtCore.Qt.ItemIsEnabled)
            self.table.item(i,3).setFlags(QtCore.Qt.ItemIsEnabled)
            self.table.item(i,4).setFlags(QtCore.Qt.ItemIsEnabled)
            self.table.item(i,5).setFlags(QtCore.Qt.ItemIsEnabled)
         

        
        self.table.resizeRowsToContents () 
        self.table.resizeColumnsToContents()

class dop_okno_opl(QtWidgets.QDialog, design_okno_opl.Ui_Dialog):

    def __init__(self,parent):
        super().__init__()
        # Это здесь нужно для доступа к переменным, методам
        # и т.д. в файле design.py
        self.setupUi(self)
        

        self.initUi()

    def initUi(self):
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2
        ex=Example()
        a=ex.label_number_chek.text()
        a=a[5:]
        oplata=[]
        cursor2=connection.cursor()
        cursor2.execute("SELECT id,price,fullname,number_kassa,kassa_ulonsaleplace_id,marka FROM kassa_basket where number_kassa="+str(a))
        for row1 in cursor2:oplata.append(row1)
        
        kolvo_alc_prod=0
        for i in range(0,len(oplata)):
            if oplata[i].get("marka")!=" ":
                kolvo_alc_prod+=1
                
        print("колво алкоголькной продук",kolvo_alc_prod)
        kolvo_kassa_ulonsaleplace_id=[]
        for i in oplata:
            kolvo_kassa_ulonsaleplace_id.append(i.get("kassa_ulonsaleplace_id"))
        c = Counter(kolvo_kassa_ulonsaleplace_id)
        
        spis_pok=c.values()
        
        spis_pok=list(spis_pok)
        c=list(c)
               
        if len(c)>=2:
            self.label.setText("Предупреждение! Оплата будет произведена в "+str(len(c))+" этапа")
            
        for i in range (0,len(c)):
            cursor.execute('SELECT shortname, address, fsrar FROM kassa_ul where id ='+str(c[i]))
            for row1 in cursor: organizac.append(row1)
        
        self.label_2.setText("Организация: "+organizac[0].get("shortname")) 
        price_pok=[]
        cursor2=connection.cursor()
        cursor2.execute("SELECT SUM(price) FROM kassa_basket where kassa_ulonsaleplace_id="+str(c[0])+" AND number_kassa="+str(a))
        for row1 in cursor2:price_pok.append(row1)
        self.label_3.setText("Итоговая сумма: "+str(price_pok[0].get("SUM(price)")))
       
    def createXML(filename):
        """
        Создаем XML файл.
        """
        root = xml.Element(u'Cheque'.encode('utf-8').decode('utf-8'),inn="7726015994".encode('utf-8').decode('utf-8'),datetime="1211123".encode('utf-8').decode('utf-8'),kpp="77240100".encode('utf-8').decode('utf-8'),kassa="45664".encode('utf-8').decode('utf-8'),address=u"г. Москва, ул. Никопольская,4".encode('utf-8').decode('utf-8'),name=u"Гармония".encode('utf-8').decode('utf-8'),number="45".encode('utf-8').decode('utf-8'),shift="1".encode('utf-8').decode('utf-8'))
        title = xml.SubElement(root,"Bottle".encode('utf-8').decode('utf-8'),barcode="16N00001CJPFO4450G71NSP20905004004797o326811691897119682191882211821".encode('utf-8').decode('utf-8'),ean="177736216338".encode('utf-8').decode('utf-8'),price="150.00".encode('utf-8').decode('utf-8'),volume="1.0000".encode('utf-8').decode('utf-8'))

        tree = xml.ElementTree(root)

        with open(filename, "wb" ) as fh:
            tree.write(fh,encoding="utf-8",xml_declaration=True)

class dop_okno_korz(QtWidgets.QDialog, design_new_dop_okno.Ui_Dialog):

    def __init__(self,parent):
        super().__init__()
        # Это здесь нужно для доступа к переменным, методам
        # и т.д. в файле design.py
        self.setupUi(self)
        

        self.initUi()

    def initUi(self):
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2
        
        #размерности экрана
        w=desktop.width()/2
        h=desktop.height()
        self.setGeometry(desktop.width()/8,desktop.height()/4, desktop.width()/4,desktop.height()/2)
        self.table1=self.tableWidget
        self.label.setText("Выберите чек")
        self.table1.setColumnCount(1)
        self.table1.setGridStyle(0)

        #выделенная строка

        self.table1.setHorizontalHeaderLabels([""])
        
         
        fnt = self.table1.font()
        fnt.setPointSize(20)

        #заголовки по центру
        name=[]
        self.table1.horizontalHeaderItem(0).setTextAlignment(Qt.AlignHCenter)
        indices = self.table1.selectionModel().currentIndex().row()
                
        header = self.table1.horizontalHeader()       
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)
        kassa=[]


        connection.commit()
        for i in range(0,len(ul)):
            cursor2=connection.cursor()
            cursor2.execute("SELECT price,number_kassa FROM kassa_basket")
        for row1 in cursor2: kassa.append(row1)
        a=0
        
        
        kassa_number=[]
        for i in kassa:
            kassa_number.append(i.get("number_kassa"))
        

        c = Counter(kassa_number)
        
        spis_pok=c.values()
        
        spis_pok=list(spis_pok)
        c=list(c)
        
            
        
        
        for i in range(0,len(c)): 
            self.table1.insertRow(i) #указательно что новую строку нужно в начало
            self.table1.setRowHeight(i,60)
            btnd = QPushButton(self)
         
            btnd.setText("                                    Чек №"+str(c[i])+"      количество позиций    "+str(spis_pok[i])+"                                                                                               ")
            
            self.table1.setCellWidget(i, 0, btnd) 
            btnd.clicked.connect(self.accept)
            #self.table1.setItem(i, 1, QTableWidgetItem(info_pokupka[i].get('fullname')))
                        
        ''' 
        for i in range(0,len(kassa)-1):
            if kassa[i].get("number_kassa")==kassa[i+1].get("number_kassa"):
                
                a+=kassa[i].get("price")
                
            elif i==len(kassa)-1:
                kassa_cen.append(a)

            else:
                kassa_cen.append(a)
                a=0
        print("цена каждой корзины",kassa_cen)
                
        
        
        
        product1=[]
        product2=[]
        for i in range(0,len(product)):
            if len(product[i].get("shtrih"))<7:
                product1.append(product[i])

        for i in range(0,len(product1)): 
            self.table1.insertRow(i) #указательно что новую строку нужно в начало
            self.table1.setRowHeight(i,60)
            btnd = QPushButton(self)
         
            btnd.setText(product1[i].get('fullname'))
            
            self.table1.setCellWidget(i, 0, btnd) 
            btnd.clicked.connect(self.accept)
            #self.table1.setItem(i, 1, QTableWidgetItem(info_pokupka[i].get('fullname')))        

      
        '''
class dop_okno_podbor(QtWidgets.QDialog, design_new_dop_okno.Ui_Dialog):

    def __init__(self,parent):
        super().__init__()
        # Это здесь нужно для доступа к переменным, методам
        # и т.д. в файле design.py
        self.setupUi(self)
        

        self.initUi()

    def initUi(self):
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2
        
        #размерности экрана
        w=desktop.width()/2
        h=desktop.height()
        self.setGeometry(desktop.width()/8,desktop.height()/4, desktop.width()/4,desktop.height()/2)
        self.table1=self.tableWidget
        self.label.setText("Выберите товар из списка.")
        self.table1.setColumnCount(2)
        self.table1.setGridStyle(0)

        #выделенная строка

        self.table1.setHorizontalHeaderLabels([' ',"                                    наименование                                             "])
        
         
        fnt = self.table1.font()
        fnt.setPointSize(20)

        #заголовки по центру
        name=[]
        self.table1.horizontalHeaderItem(0).setTextAlignment(Qt.AlignHCenter)
        indices = self.table1.selectionModel().currentIndex().row()
                
        header = self.table1.horizontalHeader()       
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)
        header.setSectionResizeMode(1, QtWidgets.QHeaderView.Stretch)
        kassa_goods_id=[]
        product=[]
        connection.commit()
        for i in range(0,len(ul)):
            cursor2=connection.cursor()
            cursor2.execute("SELECT kassa_goods_id FROM kassa_rests where kassa_ulonsaleplace_id="+str(ul[i].get('ul_id')))
            for row1 in cursor2: kassa_goods_id.append(row1)
        for i in range(0,len(kassa_goods_id)):                    
            cursor2=connection.cursor()
            cursor2.execute("SELECT id, shtrih, fullname FROM kassa_goods where alc!=1 and marka!=1 and markatovar!=1 and id="+str(kassa_goods_id[i].get('kassa_goods_id')))
            for row1 in cursor2: product.append(row1) 
        
        product1=[]
        product2=[]
        for i in range(0,len(product)):
            if len(product[i].get("shtrih"))<7:
                product1.append(product[i])


        
        
        for i in range(0,len(product1)): 
            self.table1.insertRow(i) #указательно что новую строку нужно в начало
            self.table1.setRowHeight(i,60)
            btnd = QPushButton(self)
         
            btnd.setText(product1[i].get('fullname'))
            self.table1.setItem(i, 0, QTableWidgetItem(str(product1[i].get('id'))))
            self.table1.setCellWidget(i, 1, btnd) 
            btnd.clicked.connect(self.accept)
            #self.table1.setItem(i, 1, QTableWidgetItem(info_pokupka[i].get('fullname')))
            

            self.table1.item(i, 0).setBackground(QtGui.QColor(255, 255, 205))
            self.table1.item(i, 0).setForeground(QBrush(QColor(255, 255, 205)))
                        
            self.table1.item(i,0).setFlags(QtCore.Qt.ItemIsEnabled)
            
            
            
    def zapisat(self):

        
        self.hide()
        

class dop_okno(QtWidgets.QDialog, design_new_dop_okno.Ui_Dialog):
    def __init__(self,parent,info_pokupka):
        super().__init__()
        # Это здесь нужно для доступа к переменным, методам
        # и т.д. в файле design.py
        self.setupUi(self)
        

        self.initUi(info_pokupka)

    def initUi(self,info_pokupka):
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2
        self.setGeometry(desktop.width()/8,desktop.height()/4, desktop.width()/4,desktop.height()/2)
        
        #размерности экрана
        w=desktop.width()/2
        h=desktop.height()
        self.table1=self.tableWidget
        self.label.setText("Этот штрих-код назначен на несколько товаров. Выберите товар из списка.")
        self.table1.setColumnCount(1)
        self.table1.setGridStyle(0)

        #выделенная строка

        self.table1.setHorizontalHeaderLabels(["наименование"])
         
        fnt = self.table1.font()
        fnt.setPointSize(20)

        #заголовки по центру
        name=[]
        self.table1.horizontalHeaderItem(0).setTextAlignment(Qt.AlignHCenter)
        indices = self.table1.selectionModel().currentIndex().row()
                
        header = self.table1.horizontalHeader()       
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)

        for i in range(0,len(info_pokupka)): 
            self.table1.insertRow(i) #указательно что новую строку нужно в начало
            self.table1.setRowHeight(i,60)
            btnd = QPushButton(self)
            btnd.setText(info_pokupka[i].get('fullname'))
            
            self.table1.setCellWidget(i, 0, btnd) 
            btnd.clicked.connect(self.accept)
            #self.table1.setItem(i, 1, QTableWidgetItem(info_pokupka[i].get('fullname')))
            
            
    def zapisat(self,info_pokupka):

        
        self.hide()
 
class Vvod_new_tovar(QtWidgets.QDialog):
    def __init__(self, parent):
        super().__init__(parent, QtCore.Qt.Window) 
        self.build()

    def build(self):
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2
        #appearance = self.palette()
        #appearance.setColor(QtGui.QPalette.Normal, QtGui.QPalette.Window,QtGui.QColor("white"))
        #self.setPalette(appearance)
        w=desktop.width()/2
        h=desktop.height()

        

        #self.mainLayout = QtWidgets.QVBoxLayout()
        self.setGeometry(w*3/8, h/4,w/4,h/1.6)
        self.setWindowTitle('Добавить товар')
        self.setFont(QtGui.QFont("Times", 12))
        
        w0=w*3/8
        h0=h/4
        w1=w/4
        h1=h/2

        self.hbox = QtWidgets.QVBoxLayout()
        self.vbox = QtWidgets.QHBoxLayout()
        
        
        self.vnt0=QLabel("Введите штрих код или марку",self)
        #self.vnt0.setGeometry(w0*0.05, 0.05*h0, 0.5*w0, 0.1*h0) 
        
        self.vnt1=QLabel("Введите наименование продукции",self)
        #self.vnt1.setGeometry(w0*0.05, 0.3*h0, 0.5*w0, 0.1*h0) 
        
        self.vnt2=QLabel("Наименование продукции из общего справочника",self)
        #self.vnt2.setGeometry(w0*0.05, 0.5*h0, 0.55*w0, 0.1*h0) 
        
        
        self.vnt3=QLabel("Единица измерения",self)
        #self.vnt3.setGeometry(w0*0.05, 0.75*h0, 0.55*w0, 0.1*h0) 
        
        self.vnt4=QLabel("Владелец",self)
        #self.vnt4.setGeometry(w0*0.05, h0, 0.55*w0, 0.1*h0) 
        
        self.vnt5=QLabel("Цена",self)
        #self.vnt5.setGeometry(w0*0.05, 1.25*h0, 0.5*w0, 0.1*h0)
        
        self.vnt6=QLabel("Категория товара",self)
        #self.vnt6.setGeometry(w0*0.05, 1.45*h0, 0.5*w0, 0.1*h0)
        


        #все поля для ввода
        
        self.vnt_pole0 = QLineEdit(self)
        #self.vnt_pole0.setGeometry(w0*0.05, 0.15*h0, 0.55*w0, 0.1*h0)
        
        self.vnt_pole1 = QLineEdit(self)
        #self.vnt_pole1.setGeometry(w0*0.05, 0.4*h0, 0.55*w0, 0.1*h0)
        
        self.vnt_pole2 = QLineEdit(self)
        #self.vnt_pole2.setGeometry(w0*0.05, 0.6*h0, 0.55*w0, 0.1*h0)
        
        self.vnt_pole3 = QLineEdit(self)
        #self.vnt_pole3.setGeometry(w0*0.05, 1.35*h0, 0.55*w0, 0.1*h0)
        

        #self.vnt_pole4.setGeometry(w0*0.05, 1.9*h0, 0, 0)
        
        #выпадающий список
        self.combo_vnt1 = QComboBox(self)
        #self.combo_vnt1.setGeometry(w0*0.05, 0.85*h0, 0.55*w0, 0.1*h0)
        self.combo_vnt1.addItems(["шт","л","кг"]) 
        #выпадающий список с категориями товара
        
        self.combo_vnt3 = QComboBox(self)
        #self.combo_vnt3.setGeometry(w0*0.05, 1.55*h0, 0.55*w0, 0.1*h0)
        self.combo_vnt3.addItems(kategor) 
        
        #выпадающий список со списками организаций

        self.combo_vnt2 = QComboBox(self)
        #self.combo_vnt2.setGeometry(w0*0.05, 1.1*h0, 0.55*w0, 0.1*h0)
        self.combo_vnt2.addItems(spisok_origanizac) 
        
        self.cb = QCheckBox('Маркировка', self)
        #self.cb.toggle()
        #self.cb.setGeometry(w0*0.05, 1.7*h0, 0.55*w0, 0.1*h0)
        #self.cb.stateChanged.connect(self.changeOkno)
        
        
        self.btn1_vnt = QPushButton('добавить',self)

        #self.btn1_vnt.setGeometry(w0*0.2, 2.3*h0, 0.25*w0, 0.2*h0)
        self.btn1_vnt.setStyleSheet('''
        QPushButton:hover {background-color: rgb(255, 255, 255); color: #3C6A47;} 
        QPushButton:!hover {background-color: rgb(255, 255, 255); color: #C3D6C8;} 
        ''')
        #self.btn1_vnt.setFont(QtGui.QFont("Times", 18, QtGui.QFont.Bold))
        self.btn1_vnt.clicked.connect(self.BD) #срабатывающая фукнци

        
                
        self.btn2_vnt = QPushButton('отмена',self)

        #self.btn2_vnt.setGeometry(w0*0.45, 2.3*h0, 0.25*w0, 0.2*h0)
        self.btn2_vnt.setStyleSheet('''
        QPushButton:hover {background-color: rgb(255, 255, 255); color: #3C6A47;} 
        QPushButton:!hover {background-color: rgb(255, 255, 255); color: #C3D6C8;} 
        ''')
        #self.btn2_vnt.setFont(QtGui.QFont("Times", 18, QtGui.QFont.Bold))
        self.btn2_vnt.clicked.connect(self.otmena)
        self.lbl9=QLabel(self)
        self.lbl10=QLabel(self)
        self.hbox.addWidget (self.vnt0)
        self.hbox.addWidget (self.vnt_pole0)
        #hbox.addWidget (self.lbl9)
        self.hbox.addWidget (self.vnt1)
        self.hbox.addWidget (self.vnt_pole1)
        self.hbox.addWidget (self.vnt2)
        self.hbox.addWidget (self.vnt_pole2)
        self.hbox.addWidget (self.vnt3)
        self.hbox.addWidget (self.combo_vnt1)
        self.hbox.addWidget (self.vnt4)
        self.hbox.addWidget (self.combo_vnt2)
        self.hbox.addWidget (self.vnt5)
        self.hbox.addWidget (self.vnt_pole3)
        self.hbox.addWidget (self.vnt6)
        self.hbox.addWidget (self.combo_vnt3)

        self.vnt_pole0.textChanged.connect(self.doSomething)
        
        self.vnt_pole1.textChanged.connect(self.doSomething1)
        self.vnt_pole3.textChanged.connect(self.doSomething2)
        
        self.hbox.addWidget (self.cb)
        self.vbox.addWidget (self.btn1_vnt)
        self.vbox.addWidget (self.btn2_vnt)
        
        self.hbox.addLayout (self.vbox)
        
        self.setLayout(self.hbox)
        

        
                
    def doSomething(self):
        if len(self.vnt_pole0.text())<3 or self.vnt_pole0.text().isdigit()==False :
            
            self.hbox.insertWidget(2,self.lbl9) 
            self.lbl9.setText("Минимум три символа")
            self.lbl9.setStyleSheet("color: red;")
            self.vnt_pole0.setStyleSheet("color: red;")
            self.vnt_pole0.setFont(QtGui.QFont("Times", 12))
            self.lbl9.setFont(QtGui.QFont("Times", 12))
        else:
            self.vnt_pole0.setStyleSheet("color: black;")
            self.vnt_pole0.setFont(QtGui.QFont("Times", 12))
            
            
            if self.lbl9 != None:
                self.lbl9.clear()
                self.hbox.removeWidget(self.lbl9)
                sip.delete(self.lbl9)
                self.lbl9 = None
            self.lbl9=QLabel(self)
            self.lbl9.setFont(QtGui.QFont("Times", 12))
            

    
    def doSomething1(self):
        a=self.vnt_pole1.text().lower()
        #spisok_zapr=["пив","водка","ликёр","вино","винн","сигарет","коньяк","водка"]
        if a.find("пив")!=-1 or a.find("водка")!=-1 or a.find("винн")!=-1 or a.find("вино")!=-1 or a.find("сигарет")!=-1 or a.find("настойка")!=-1 or a.find("ликёр")!=-1  or a.find("коньяк")!=-1 or a.find("виски")!=-1 or a.find("ром")!=-1:
            self.hbox.insertWidget(4,self.lbl10)
            self.lbl10.setText("Наоминание! Так вы можете добавить только НЕ алколольную продукцию")
            self.vnt_pole1.setStyleSheet("color:  red;")
            self.vnt_pole1.setFont(QtGui.QFont("Times", 12))
            self.lbl10.setStyleSheet("color: red;")
            self.lbl10.setFont(QtGui.QFont("Times", 12))
        else:
            self.vnt_pole1.setStyleSheet("color: black;")
            self.vnt_pole1.setFont(QtGui.QFont("Times", 12))

            if self.lbl10 != None:
                self.lbl10.clear()
                self.hbox.removeWidget(self.lbl10)
                sip.delete(self.lbl10)
                self.lbl10 = None
            self.lbl10=QLabel(self)
            self.lbl10.setFont(QtGui.QFont("Times", 12))

            
    def doSomething2(self):
        if self.vnt_pole3.text().isdigit()==False:
            self.vnt_pole3.setStyleSheet("background-color: pink;")
            self.vnt_pole3.setFont(QtGui.QFont("Times", 12))
        else:
            self.vnt_pole3.setStyleSheet("background-color: white;")
            self.vnt_pole3.setFont(QtGui.QFont("Times", 12))

            
            
    """
    функция автоматического заполнения при обнаружении товара
    """

    def dobavlenie_new_tovar(self):
        connection.commit()
        cursor=connection.cursor()
        info_pokupka=[]
        price=[]
        if cursor.execute("SELECT name FROM egais_allshtrih where shtrih="+str(self.vnt_pole0.text()))!=0:
            for row1 in cursor: info_pokupka.append(row1)
            self.vnt_pole1.setText(info_pokupka[0].get("name"))
            self.vnt_pole2.setText(info_pokupka[0].get("name"))
            self.vnt_pole3.setFocus(0)

    def BD(self,state):
        if self.vnt_pole0.text()!='' and self.vnt_pole1.text()!='' and self.vnt_pole3.text()!='':
            connection.commit()
            cursor=connection.cursor()
            id_kategor=[]
            cursor.execute("SELECT id FROM kassa_categ where name= '"+self.combo_vnt3.currentText()+"'" )
            for row1 in cursor: id_kategor.append(row1)
            id_org=ul[spisok_origanizac.index(self.combo_vnt2.currentText())]

            ed_izm=[["шт","л","кг"],[0,1,2],["ed","l","kg"]]
            typ_id=ed_izm[1][ed_izm[0].index(self.combo_vnt1.currentText())]
            edizm_id=ed_izm[2][ed_izm[0].index(self.combo_vnt1.currentText())]
            if self.cb.isChecked():
                markatovar=1
            else: markatovar=0
            
            #запись в БД
            cursor=connection.cursor()
            cursor.execute("""

            INSERT INTO kassa_goods (fullname,fullname2,categ_id,shtrih,kod,alc,typ_id,edizm_id,
            maker_ulid,marka,nalog,mercury,sync,shtrih2,markatovar)values ('"""+str(self.vnt_pole1.text())+
            "',''," +str(id_kategor[0].get("id"))+","+str(self.vnt_pole0.text())+",'','0'," +str(typ_id)+",'"+
            str(edizm_id)+"','0','0','0','0','0','',"+str(markatovar)+")")
            
            connection.commit()

            max_id=[]
            cursor=connection.cursor()
            cursor.execute("SELECT MAX(ID)FROM KASSA_GOODS")
            for row1 in cursor: max_id.append(row1)
            print("!!",max_id)
                
                
            max_id_kp=[]
            cursor=connection.cursor()
            cursor.execute("SELECT MAX(ID)FROM KASSA_price")
            for row1 in cursor: max_id_kp.append(row1)
            #получение даты
            today = datetime.datetime.today()
            a= today.strftime("%Y-%m-%d %H:%M:%S")
            
            cursor=connection.cursor()
            
            cursor.execute("""
            INSERT INTO kassa_price (dt,kassa_wb_id,price,kassa_goods_id)values ('"""+
            str(a)+
            "','0'," +str(self.vnt_pole3.text())+","+str(max_id[0].get("MAX(ID)"))+")")
            
            cursor=connection.cursor()
            cursor.execute("""
            INSERT INTO kassa_rests (quantity,kassa_price_id,kassa_ulonsaleplace_id,kassa_goods_id)values ("""
            +"'-1',"+str(+max_id_kp[0].get("MAX(ID)"))+","+str(id_org.get("ul_id"))+","+
            str(max_id[0].get("MAX(ID)"))+")")
            connection.commit()

            connection.commit()
            self.vnt_pole0.clear()
            self.vnt_pole1.clear()
            self.vnt_pole2.clear()
            self.vnt_pole3.clear()

            self.switch()
        else:
            if self.vnt_pole0.text()=='':
                self.vnt_pole0.setStyleSheet("background-color: pink;")
            if self.vnt_pole1.text()=='':
                self.vnt_pole1.setStyleSheet("background-color: pink;")
            if self.vnt_pole3.text()=='':
                self.vnt_pole3.setStyleSheet("background-color: pink;")
                
            

        

    def switch(self):
        self.hide()
        info_pokupka=[]
        cursor=connection.cursor()
        L=[]
        connection.commit()
        cursor=connection.cursor()
        cursor.execute("""SELECT max(id) from kassa_goods""")
        for row1 in cursor: L.append(row1)
        cursor=connection.cursor()
        cursor.execute("SELECT id,shtrih,fullname,fullname2 FROM kassa_goods where id="+str(L[0].get("max(id)")))    
        for row1 in cursor: info_pokupka.append(row1)


        info_pokupka[0].get('fullname')
        price=[]
        cursor1=connection.cursor()
        cursor1.execute("SELECT id,price FROM kassa_price where kassa_goods_id="+str(info_pokupka[0].get('id')))
        for row1 in cursor1: price.append(row1)
        quantity=[]
        cursor2=connection.cursor()
        cursor2.execute("SELECT id,quantity,kassa_ulonsaleplace_id FROM kassa_rests where kassa_goods_id="+str(info_pokupka[0].get('id')))
        for row1 in cursor2:quantity.append(row1)
        pole="000001111"
        self.accept()

    #при нажатии на кнопку отмена очистка полей
    def otmena(self):

        self.vnt_pole0.clear()
        self.vnt_pole1.clear()
        self.vnt_pole2.clear()
        self.vnt_pole3.clear()

   
        self.cls_w()


    def keyReleaseEvent(self, e):
        #нажатие на кнопку enter автоматическое заполенние, если возможно
        if (e.key()==16777220 ) and str(self.vnt_pole0.text())!='':
            print("нажат!!!")
            self.dobavlenie_new_tovar()
 

    #закрыть окно
    def cls_w(self):
        

        self.close()

"""
Окно кассы
"""


class Example(QtWidgets.QMainWindow, design_new.Ui_MainWindow):

    def __init__(self):
        # Это здесь нужно для доступа к переменным, методам
        # и т.д. в файле design.py
        super().__init__()
        
        self.setupUi(self)  # Это нужно для инициализации нашего дизайна
        self.secondWin = None
        self.vvodnewtovarWin = None
        self.dop_oknoWin = None
        self.gurnal_prWin = None

        self.initUI()

 

    def initUI(self):
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2

        #размерности экрана
        w=desktop.width()/2
        h=desktop.height()
        self.setGeometry(0, 0, desktop.width()/2,desktop.height())
        self.setWindowTitle('ОБЛАКО')
        #список элементов!

        self.table = self.tableWidget 
        self.qle=self.lineEdit
        self.combo=self.comboBox
        
        self.btn1_tovar =self.pushButton
        self.btn1_tovar.clicked.connect(self.tovar)
        
        self.btn2_n_tovar =self.pushButton_2
        self.btn2_n_tovar.clicked.connect(self.open_dialog_3)
        
        self.btn_dop_kn =self.pushButton_3
        self.btn_dop_kn.clicked.connect(self.dop_kn)
       
        
        self.btn_podbor =self.pushButton_4
        self.btn_podbor.clicked.connect(self.open_dialog2)
        
        self.btn_delete_chek=self.pushButton_5
        self.btn_delete_chek.clicked.connect(self.open_dialog_4)
        
        
        
        self.btn_opl=self.pushButton_6
        self.btn_opl.clicked.connect(self.open_dialog_5)
        
        self.btn_otl_chek=self.pushButton_7
        self.btn_otl_chek.clicked.connect(self.new_kassa_chek_butt)

        self.label_number_chek=self.label_4

        self.qle1 = QLineEdit(self)
        self.qle1.setGeometry(0,0,0,0)
        
        

        #self.qle1=self.lineEdit

        self.label.setText(
            """
            Общество с ограниченной ответственностью "Тансу регион 2" 020000838105     Татарстан Респ, Зеленодольский 
            р-н, Зеленодольск г, , Гагарина ул, 5/2, ЮГАН-АЛКО 020000148280     Татарстан Респ, Нижнекамский р-н, , Камские Поляны пгт, , 1/19 "г"
            """ )
    
        now = QDate.currentDate() 
        self.label_2.setText(now.toString(Qt.ISODate))
        self.label_3.setText("кассир:")
        self.lbl9=QLabel(self)
        spisok_prodavc.append("ДОБАВИТЬ КАССИРА")
        self.comboBox.addItems(spisok_prodavc) 
        self.combo.activated.connect(self.combo_new_kassir)
        #self.lbl4=self.label_4
        self.label_4.setText("Чек №1")
        
        #self.btn_udal = self.pushButton_7

        #self.btn_udal.clicked.connect(self.otmena_vvoda)
        
        timer = QTimer(self)
        timer.timeout.connect(self.showTime)
        timer.start(1000)
        self.showTime() 
        self.table.setColumnCount(8)
        self.table.setGridStyle(0) 
        #self.qle1=QLineEdit(self)
        self.qle.setFocus(0)
        self.lbl9.setStyleSheet("color:#FF0000;")
        self.lbl9.setFont(QtGui.QFont("Times", 25))

        #self.lbl6=QLabel(self)
        #self.lbl6.setStyleSheet("color:#FF0000;")
        #self.lbl6.setFont(QtGui.QFont("Times", 25))
        
        self.table.cellChanged.connect(self.onCellChanged)
        #self.tableWidget_2.deleteLater()
        #self.lineEdit_2.deleteLater()
        A=[]
        connection.commit()
        cursor1=connection.cursor()
        cursor1.execute("SELECT COUNT(*) FROM kassa_basket")
        for row1 in cursor1: A.append(row1)


                

        self.table.cellChanged.connect(self.onCellChanged)


        # Обработчик сигнала
        #self.btn4.clicked.connect(lambda: processing(self.my_signal))
        number_chek=[]
        number_chek1=[]
        if self.internet_on()==False:

            self.label_6.setText("НЕТ ИНТЕРНЕТА")
           
            

        if A[0].get('COUNT(*)')!=0:
            connection.commit()
            cursor1=connection.cursor()
            cursor1.execute("SELECT number_kassa FROM kassa_basket")
            for row1 in cursor1: number_chek.append(row1)
            for i in number_chek:
                number_chek1.append(i.get("number_kassa"))
            number_chek1=list(set(number_chek1))
            self.zapolnenie(number_chek1[0])
            self.label_number_chek.setText("Чек №"+str(number_chek1[0]))
            

                
        else:
            self.table.setHorizontalHeaderLabels(["","НАИМЕНОВАНИЕ", "КОЛ-ВО"," ","  ЦЕНА   "," ОСТАТОК ","   СУММА   "," "])
            fnt = self.table.font()
            fnt.setPointSize(40)
            #заголовки по центру
            for i in range(0,8):
                self.table.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            self.label_5.setText("итого: 0 "+'Р')
            self.label_number_chek.setText("Чек №1")
            
            
        
         



    def internet_on(self):
        try:
            response=urllib.request.urlopen('http://vk.com',timeout=40)
            return True
        except urllib.request.URLError as err: pass
        return False
              

        
    def showTime(self):
        time = QTime.currentTime()
        text = time.toString('hh:mm')
        if (time.second() % 2) == 0:
            text = text[:2] + ' ' + text[3:]

        self.lcdNumber.display(text)
    def combo_new_kassir(self):
        if self.combo.currentText()=="ДОБАВИТЬ КАССИРА":

            text, ok = QInputDialog.getText(self, 'Новый кассир',
                'Введите имя:')

            if ok:
                cursor=connection.cursor()
                cursor.execute("""

                INSERT INTO kassa_kassir (fio,current)values ('"""+str(text)+"','0')")

                
                connection.commit()
                self.combo.clear()
                
                spisok_prodavc.insert(0,text)
                self.comboBox.addItems(spisok_prodavc) 
                

          
            
        
    '''
        
    def new_kassa_chek(self,*args):
        connection.commit()
        cursor1=connection.cursor()
        number_chek=[]
        number_chek1=[]
        cursor1.execute("SELECT number_kassa FROM kassa_basket")
        for row1 in cursor1: number_chek.append(row1)
        for i in number_chek:
            number_chek1.append(i.get("number_kassa"))
        number_chek1=list(set(number_chek1))
        
        print(str(*args) )  
        if str(*args)=='':
            btn_kassa = QPushButton(str(int(self.verticalLayout_4.itemAt(self.verticalLayout_4.count()-1).widget().text())+1),self)
        #self.btn1_vnt.setGeometry(w0*0.2, 2.3*h0, 0.25*w0, 0.2*h0)
        else: 
            btn_kassa = QPushButton(str(*args),self)

        #self.btn1_vnt.setFont(QtGui.QFont("Times", 18, QtGui.QFont.Bold))
        self.verticalLayout_4.insertWidget(-1,btn_kassa) 
        a=btn_kassa.text()
        btn_kassa.clicked.connect(lambda:self.new_kassa_chek_butt_1(a, btn_kassa)) #срабатывающая фукнци
        self.new_kassa_chek_butt(int(a))
                


        
        
    def new_kassa_chek_butt_1(self,numb, btn_kassa):
        #numb=self.btn_kassa.text()
        self.label_number_chek.setText("Чек №"+str(numb))       
        
        c=0
        for i in range(self.verticalLayout_4.count()):
            c+=1

        
        position=[]
        number_chek1=[]
        connection.commit()
        cursor1=connection.cursor()
        cursor1.execute("SELECT COUNT(*) FROM kassa_basket WHERE NUMBER_KASSA="+str(numb))
        for row1 in cursor1: position.append(row1)
        for i in position:
            number_chek1.append(i.get("number_kassa"))
        number_chek1=list(set(number_chek1))                

            
                    
        for i in range(self.verticalLayout_4.count()):   
             #layout.itemAt(i).widget().deleteLater()    
            if self.verticalLayout_4.itemAt(i).widget().text()==self.label.text()[5:]:
            
                print("название кнопки",numb)
                print("индекс",i)
                effect = QGraphicsDropShadowEffect( self.verticalLayout_4.itemAt(i).widget() )
                effect.setOffset(0, 0)
                effect.setBlurRadius(20)
                self.verticalLayout_4.itemAt(i).widget().setGraphicsEffect(effect)
                
        for i in range(self.verticalLayout_4.count()):   
             #layout.itemAt(i).widget().deleteLater()    
            if self.verticalLayout_4.itemAt(i).widget().text()!=self.label.text()[5:]:
            
                print("название кнопки",numb)
                print("индекс",i)
                effect = QGraphicsDropShadowEffect( self.verticalLayout_4.itemAt(i).widget() )
                effect.setOffset(0, 0)
                effect.setBlurRadius(0)
                self.verticalLayout_4.itemAt(i).widget().setGraphicsEffect(effect)                
                
                
        if position[0].get('COUNT(*)')!=0:
                self.table.setRowCount(0)
                self.zapolnenie(numb)
                self.table.cellChanged.connect(self.onCellChanged)
                
               
        else: 

            
            self.table.setRowCount(0)
            self.table.cellChanged.connect(self.onCellChanged)
            
     
    def delete_korsina(self):
        a=self.label_number_chek.text()
        a=a[len(a)-1]

        #удаление из корзины по id 
        cursor=connection.cursor()
        cursor.execute("""
        DELETE FROM kassa_basket WHERE number_kassa="""+str(a))
        connection.commit()
        self.table.setRowCount(0)
        self.qle.setFocus(0)
        
 
        spisok_kass=[]
    
        for i in range(self.verticalLayout_4.count()): 
            print(i)
            #layout.itemAt(i).widget().deleteLater()    
            if self.verticalLayout_4.itemAt(i).widget().text()==self.label.text()[5:]:
                effect = QGraphicsDropShadowEffect( self.verticalLayout_4.itemAt(i).widget() )
                effect.setOffset(0, 0)
                effect.setBlurRadius(0)
                self.verticalLayout_4.itemAt(i).widget().setGraphicsEffect(effect)    
                

                #effect = QGraphicsDropShadowEffect( self.verticalLayout_4.itemAt(i).widget() )
                self.verticalLayout_2.removeWidget(self.verticalLayout_4.itemAt(i).widget())
                sip.delete(self.verticalLayout_4.itemAt(i).widget())
                if self.verticalLayout_4.itemAt(0)!=None: 
                    self.new_kassa_chek_butt(self.verticalLayout_4.itemAt(0).widget().text())
                else:
                    self.label_5.setText("итого: 0 "+'Р')
                    self.new_kassa_chek(1)

                #self.verticalLayout_4.itemAt(i).widget() = None
                break
                


  
        print("список касса",self.verticalLayout_4.count())       
        self.itogo()
        
        
    '''  
    def dop_kn(self):
        #self.btn_dop_kn.setEnabled(False) 
        self.btn_dop_kn.hide()
        print("1 фукнция")
        self.vozvrat = QtWidgets.QPushButton(self.centralwidget)
        self.vozvrat.setStyleSheet("font: 14pt \"Microsoft YaHei\";\n"
        "background-color: rgb(255, 214, 143);")
        self.horizontalLayout_8.addWidget(self.vozvrat)
        self.vozvrat.setText("возврат")
        
        
        self.vnesenie = QtWidgets.QPushButton(self.centralwidget)
        self.vnesenie.setStyleSheet("font: 14pt \"Microsoft YaHei\";\n"
        "background-color: rgb(255, 214, 143);")
        self.horizontalLayout_8.addWidget(self.vnesenie)
        self.vnesenie.setText("внесение")
        
        self.viemka = QtWidgets.QPushButton(self.centralwidget)
        self.viemka.setStyleSheet("font: 14pt \"Microsoft YaHei\";\n"
        "background-color: rgb(255, 214, 143);")
        self.horizontalLayout_8.addWidget(self.viemka)
        self.viemka.setText("выемка")
        
        
        self.x_otchet = QtWidgets.QPushButton(self.centralwidget)
        self.x_otchet.setStyleSheet("background-color: rgb(255, 214, 143);\n"
        "font: 14pt \"Microsoft YaHei\";\n""")
        self.horizontalLayout_8.addWidget(self.x_otchet)
        self.x_otchet.setText("X отчёт")
        
        
        self.z_otchet = QtWidgets.QPushButton(self.centralwidget)
        self.z_otchet.setStyleSheet("font: 14pt \"Microsoft YaHei\";\n"
        "background-color: rgb(255, 214, 143);")
        self.horizontalLayout_8.addWidget(self.z_otchet)
        self.z_otchet.setText("Z отчёт")
        
        self.g_pr = QtWidgets.QPushButton(self.centralwidget)
        self.g_pr.setStyleSheet("font: 14pt \"Microsoft YaHei\";\n"
        "background-color: rgb(255, 214, 143);")
        self.horizontalLayout_8.addWidget(self.g_pr)
        self.g_pr.setText("журнал продаж")
        self.g_pr.clicked.connect(self.open_dialog_6)
        
        self.skr = QtWidgets.QPushButton(self.centralwidget)
        self.skr.setStyleSheet("font: 14pt \"Microsoft YaHei\";\n"
        "background-color: rgb(255, 214, 143);")
        self.horizontalLayout_8.addWidget(self.skr)
        self.skr.setText("скрыть") 
        self.skr.clicked.connect(self.zakr_okno)
        
        
        
    
    def zakr_okno(self):

        
        if self.vozvrat != None:      
            print("СРАБОТАЛО")
            self.horizontalLayout_8.removeWidget(self.vozvrat )
            sip.delete(self.vozvrat)
            self.vozvrat = None
            
        if self.vnesenie != None:    
            self.horizontalLayout_8.removeWidget(self.vnesenie )
            sip.delete(self.vnesenie)
            self.vnesenie = None     
            
        if self.viemka != None:
            self.horizontalLayout_8.removeWidget(self.viemka )
            sip.delete(self.viemka)
            self.viemka = None
            
            
        if self.g_pr != None:
            self.horizontalLayout_8.removeWidget(self.g_pr )
            sip.delete(self.g_pr)
            self.g_pr = None
            
        if self.x_otchet != None:
            self.horizontalLayout_8.removeWidget(self.x_otchet )
            sip.delete(self.x_otchet)
            self.x_otchet = None        
        if self.z_otchet != None:
            self.horizontalLayout_8.removeWidget(self.z_otchet )
            sip.delete(self.z_otchet)
            self.z_otchet = None        
            
            
        if self.skr != None:
            self.horizontalLayout_8.removeWidget(self.skr)
            sip.delete(self.skr)
            self.skr = None    
            
        self.btn_dop_kn.show()    
        #self.btn_dop_kn.clicked.connect(self.dop_kn)

     
        
        
        
    def new_kassa_chek_butt(self):
        #numb=self.btn_kassa.text()
        kassa=[]
        for i in range(0,len(ul)):
            cursor2=connection.cursor()
            cursor2.execute("SELECT price,number_kassa FROM kassa_basket")
        for row1 in cursor2: kassa.append(row1)
        a=0
        
        
        kassa_number=[]
        for i in kassa:
            kassa_number.append(i.get("number_kassa"))
            
        self.label_number_chek.setText("Чек №"+str(max(kassa_number)+1))

        self.table.setRowCount(0)
        self.table.cellChanged.connect(self.onCellChanged)
        
    def tovar(self):
        desktop=QtWidgets.QApplication.desktop()
        w=desktop.width()/2
        h=desktop.height()
        
        #self.table.setGeometry(0, 0.3*h,w,0.4*h)
        self.table.setHorizontalHeaderLabels(["","НАИМЕНОВАНИЕ", "КОЛИЧЕСТВО","   ЦЕНА   "," ОСТАТОК ","   СУММА   "," "])
        fnt = self.table.font()
        fnt.setPointSize(40)
        #заголовки по центру
        for i in range(0,7):
            self.table.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)


        #колонки по размерности текста
        header = self.table.horizontalHeader()       
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)
        #колонка названия товара растягивается максимально
        header.setSectionResizeMode(1, QtWidgets.QHeaderView.Stretch)
        for i in range(2,8):
            header.setSectionResizeMode(i, QtWidgets.QHeaderView.ResizeToContents)


        indices = int(self.table.selectionModel().currentIndex().row())
        connection.commit()
        cursor=connection.cursor()
        info_pokupka=[]
        cursor.execute("SELECT id,shtrih,fullname,fullname2,edizm_id,marka FROM kassa_goods where shtrih=00000011111")
        for row1 in cursor: info_pokupka.append(row1)
        

        info_pokupka[0].get('fullname')
        price=[]
        cursor1=connection.cursor()
        cursor1.execute("SELECT id,price FROM kassa_price where kassa_goods_id="+str(info_pokupka[0].get('id')))
        for row1 in cursor1: price.append(row1)
        quantity=[]
        cursor2=connection.cursor()
        cursor2.execute("SELECT id,quantity,kassa_ulonsaleplace_id FROM kassa_rests where kassa_goods_id="+str(info_pokupka[0].get('id')))
        for row1 in cursor2:quantity.append(row1)
        pole="000001111"
        #установка фокуса
        #self.qle.setFocus(0)
        #добавление в таблицу
        self.new_stroka(info_pokupka,quantity,pole,'0',price[0].get("id")," ","0",[])
        


    #подсчёт суммы по одному товару
    def summa(self):
        indices =  int(self.table.selectionModel().currentIndex().row()) #выделенная строка
       
        if indices>=0 and  self.table.item(indices,2)!=None and self.table.item(indices,2).text()!='' and self.table.item(indices,4).text()!='':
            #расчёт цены колво*цену
            print("СЧИТАЕМ",str(float(self.table.item(indices,2).text())*float(self.table.item(indices,4).text())))
            self.table.setItem(indices, 6, QTableWidgetItem(str(float(self.table.item(indices,2).text())*float(self.table.item(indices,4).text()))))
            self.table.item(indices,5).setFlags(QtCore.Qt.ItemIsEnabled)
            
            
        
        '''
          
        if indices>=0 and  self.table.item(indices,2).text()!='':    
            #изменение цвета при некорректном вводе
            if float(self.table.item(indices,2).text() )<=0 or len(self.table.item(indices,2).text() )>=6 :
            #расчёт цены колво*цену :
            
                self.table.item(indices, 2).setForeground(QBrush(QColor(231, 131, 131)))
            else: 
                self.table.item(indices, 2).setForeground(QBrush(QColor(255, 255, 255)))

            if float(self.table.item(indices,4).text() )<=0 or len(self.table.item(indices,4).text() )>=6 or self.table.item(indices,4).text()=='':
                self.table.item(indices,4)..setForeground(QBrush(QColor(231, 131, 131)))
            else: 
                self.table.item(indices, 4).setForeground(QBrush(QColor(255, 255, 255)))
                
        elif self.table.item(indices,2).text()=='':
                
            self.table.item(indices, 2).setForeground(QBrush(QColor(231, 131, 131)))
        elif self.table.item(indices,2).text()!='': 
            self.table.item(indices, 2).setForeground(QBrush(QColor(255, 255, 255))) 
            
        elif self.table.item(indices,4).text()=='':            
            self.table.item(indices, 4).setForeground(QBrush(QColor(231, 131, 131)))
        else: 
            self.table.item(indices, 4).setForeground(QBrush(QColor(255, 255, 255)))           
        
               
            if self.table.item(indices,5).text()!="---" :
                
                if float(self.table.item(indices,5).text())<float(self.table.item(indices,2).text()):
                    self.table.item(indices,2).setForeground(QBrush(QColor(231, 131, 131)))
                else: 
                    
                    self.table.item(indices, 2).setForeground(QBrush(QColor(255, 255, 255)))
             
             
    '''
    #подсчёт итоговой суммы корзины
    def itogo(self):
        a=0
        rowPosition = self.table.rowCount()

        for i in range(0,rowPosition):
            #сумммируем всё что есть
            a+=float(self.table.item(i,6).text())
        #self.lbl5.setText(str(a)+' '+'Р')
        self.label_5.setText("      Итого:  "+str(a)+' '+'Р       ')
        
    def shtrih_kod(self):
        print("ФУНКЦИЯ ШТРИХ КОДА")
        desktop=QtWidgets.QApplication.desktop()
        w=desktop.width()/2
        h=desktop.height()
        self.lbl6=QLabel(self)
        self.lbl6.setStyleSheet("color:#FF0000;")
        self.lbl6.setFont(QtGui.QFont("Times", 25))
        
        

        self.table.setHorizontalHeaderLabels(["","НАИМЕНОВАНИЕ", "КОЛИЧЕСТВО"," ","   ЦЕНА   "," ОСТАТОК ","   СУММА   "," "])
        for i in range(0,8):
            self.table.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)


        #колонки по размерности текста
        header = self.table.horizontalHeader()       
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)
        #колонка названия товара растягивается максимально
        header.setSectionResizeMode(1, QtWidgets.QHeaderView.Stretch)
        for i in range(2,8):
            header.setSectionResizeMode(i, QtWidgets.QHeaderView.ResizeToContents)
        pole=str(self.qle.text())
        #словарь если отсканировали при русской расскладке, транслит текста
        layout = dict(zip(map(ord, "йцукенгшщзхъфывапролджэячсмитьбю.ё"
                           'ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ,Ё'),
                            "qwertyuiop[]asdfghjkl;'zxcvbnm,./`"
                           'QWERTYUIOP{}ASDFGHJKL:"ZXCVBNM<>?~'))
        pole=pole.translate(layout)

        quantity=[]
        #если длинны поля 68-это марка
        if len(pole)==68:
            self.marka(pole)
        #если дляинна поля 150-это марка
        elif len(pole)==150:
            self.marka(pole)
        #иначе штрих код
        elif len(pole)>=3 and pole.isdigit()==True:
            
            cursor=connection.cursor()
            connection.commit()
            info_pokupka=[]
            #если не нашли в бд запись с таким штрих кодом, товар отсутствует
            if cursor.execute("SELECT id,fullname,alc,shtrih,fullname2,edizm_id,marka,markatovar FROM kassa_goods where shtrih="+str(pole))==0:
                self.tovar_ots(w,h)
                
                
            else:
                #заполняем в список информацию о покупке
                for row1 in cursor: info_pokupka.append(row1)
                if len(info_pokupka)>1:
                    self.open_dialog(info_pokupka)
                else:
                    self.alone_shtrich(info_pokupka)
                    
        elif len(pole)!=0 and len(pole)<3 or pole.isdigit()==False:
            self.tovar_never_vvod(w,h)
            

       #если ввдено 0 1 2 символа неверный ввод                     

       
            
    def alone_shtrich(self,info_pokupka):
        print("ФУНКЦИЯ ЕДНСТВЕННЫЙ ШТРИХ КОД")
        self.lbl6=QLabel(self)
        self.lbl6.setStyleSheet("color:#FF0000;")
        self.lbl6.setFont(QtGui.QFont("Times", 25))
        desktop=QtWidgets.QApplication.desktop()
        w=desktop.width()/2
        h=desktop.height()
        rozliv=[]
        quantity=[]
        #смотрим информацию о остатках
        times=[]
        cursor2=connection.cursor()   
        cursor2.execute("SELECT begintime,endtime FROM egais_config")
        for row1 in cursor2:times.append(row1)
                
        time = datetime.datetime.now().time()
        value=times[0].get("begintime")
        time_morning=(datetime.datetime.min + value).time()
        value=times[0].get("endtime")
        time_evn=(datetime.datetime.min + value).time()
        #last_night = datetime.time(str(times[0].get("begintime"))[:2],str(times[0].get("begintime"))[3:5])
        
        connection.commit()
        cursor2=connection.cursor()
        for i in ul:
            cursor2.execute("SELECT id,quantity,kassa_ulonsaleplace_id FROM kassa_rests where kassa_ulonsaleplace_id="+str(i.get("ul_id"))+" and kassa_goods_id="+str(info_pokupka[0].get('id')))
            for row1 in cursor2:quantity.append(row1)
        #если список пустой, то нет информации о колве
        if quantity==[]:
            self.tovar_net_kolvo(w,h)
           
        #если остатки нулевые-товар отсутсвует
        elif quantity[0].get('quantity')==0.0:
            #self._update_timer =  QBasicTimer()
            print("НАЛИЧИЕ ТОВАРА")
            self.tovar_nalich(w,h)
            #self.tovar_ots(w,h)

        
        
        elif info_pokupka[0].get("alc")==1 and time<time_morning or info_pokupka[0].get("alc")==1 and time>time_evn :
            self.tovar_alc_time()
            

           

        else:
                        
            if info_pokupka[0].get('marka')==1:
                #self.hbox.insertLayout (3,self.vbox8)
                self.lineEdit_2 = QtWidgets.QLineEdit(self.centralwidget)
                self.lineEdit_2.setStyleSheet("font: 8pt \"Microsoft YaHei\";\n" "background-color: rgb(255, 255, 205);")
                    
                self.lineEdit_2.setAlignment(QtCore.Qt.AlignLeading|QtCore.Qt.AlignLeft|QtCore.Qt.AlignTop)
                self.lineEdit_2.setObjectName("lineEdit_2")
                self.verticalLayout.insertWidget(6,self.lineEdit_2)
                self.qle1=self.lineEdit_2
                            
                self.qle.setDisabled(True)
                self.qle1.setFocus(0)

            #если есть товарная марка открываем поле для ввода товарной марки
            elif info_pokupka[0].get('markatovar')==1:
                self.lineEdit_2 = QtWidgets.QLineEdit(self.centralwidget)
                self.lineEdit_2.setStyleSheet("font: 8pt \"Microsoft YaHei\";\n" "background-color: rgb(255, 255, 205);")
                self.lineEdit_2.setAlignment(QtCore.Qt.AlignLeading|QtCore.Qt.AlignLeft|QtCore.Qt.AlignTop)
                self.lineEdit_2.setObjectName("lineEdit_2")
                self.verticalLayout.addWidget(6,self.lineEdit_2)
                self.qle1=self.lineEdit_2
                            
                self.qle.setDisabled(True)
                self.qle1.setFocus(0)
            else: 
                #находим цену товара
                price=[]
                cursor1=connection.cursor()
                pole=str(self.qle.text())
                cursor1.execute("SELECT id,price FROM kassa_price where kassa_goods_id="+str(info_pokupka[0].get('id')))
                for row1 in cursor1: price.append(row1)
                #если цена не назначена пишем это
                if price==[]:
                    #self.tovar_net_cena(w,h)
                    self.new_stroka(info_pokupka,quantity,pole,'0',' ',' ','0',[])
                #добавление в таблицу    
                else:

                    self.new_stroka(info_pokupka,quantity,pole,str(price[0].get('price')),price[0].get("id"),' ',price[0].get("price"),[])
                                

    #функция добавление записи в таблицу и в бд
    def new_stroka(self,info_pokupka,quantity,pole,price_tov,price,qle1,cena,rozliv):
        a=self.label_number_chek.text()
        a=a[len(a)-1]
        print("номер кассы",a)
        desktop=QtWidgets.QApplication.desktop()
        w=desktop.width()/2
        h=desktop.height()
        self.zapis_BD(info_pokupka,quantity,price,qle1,cena)
        #выделенная строка
        #self.table.setGeometry(0, 0.3*h,w,0.4*h)
        self.table.setHorizontalHeaderLabels(["","НАИМЕНОВАНИЕ", "КОЛ-ВО","-","  ЦЕНА   "," ОСТАТОК ","   СУММА   "," "])
        fnt = self.table.font()
        fnt.setPointSize(40)
        #заголовки по центру
        for i in range(0,8):
            self.table.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)

        if info_pokupka[0].get('edizm_id')=="ed":
            ed_izm="шт    "
                
        if info_pokupka[0].get('edizm_id')=="l":
            ed_izm="л    "
        if info_pokupka[0].get('edizm_id')=="kg":
            ed_izm="кг    "
        #колонки по размерности текста
        header = self.table.horizontalHeader()       
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)
        #колонка названия товара растягивается максимально
        header.setSectionResizeMode(1, QtWidgets.QHeaderView.Stretch)
        for i in range(2,8):
            header.setSectionResizeMode(i, QtWidgets.QHeaderView.ResizeToContents)

        indices = int(self.table.selectionModel().currentIndex().row())
        rowPosition = self.table.rowCount()
        self.table.insertRow(0) #указательно что новую строку нужно в начало
        #кнопка удаления товара
        self.table.setSelectionBehavior(QAbstractItemView.SelectRows)
        btn1 = QPushButton(self)

        #кнопку в седбмой столбец
        self.table.setCellWidget(0, 7, btn1)
        self.table.setRowHeight(0,60)
        
        
        combo_v = QComboBox(self)
        #self.combo_vnt1.setGeometry(w0*0.05, 0.85*h0, 0.55*w0, 0.1*h0)
        
        
        rozliv_volume=[]
        #название
        self.table.setItem(0, 1, QTableWidgetItem(info_pokupka[0].get('fullname')+'\n'+str(info_pokupka[0].get('shtrih'))+'\n'+str(qle1)))
        #self.table.setItem(0, 1, QTableWidgetItem(info_pokupka[0].get('fullname')))
        #колво
        if rozliv==[]:
            self.table.setItem(0, 2, QTableWidgetItem("1"))
        else:
            for i in rozliv:
                rozliv_volume.append(str(i.get("volume")))
                
            combo_v.addItems(rozliv_volume)       
            self.table.setCellWidget(0, 2, combo_v)
            combo_v.currentTextChanged.connect(lambda:self.on_combobox_changed(rozliv,combo_v))
        #ед.изм
        if rozliv==[]:
            self.table.setItem(0, 3, QTableWidgetItem(ed_izm))
        else:
            self.table.setItem(0, 3, QTableWidgetItem(rozliv[0].get("edizmcode")))
            
        #цена
        if rozliv==[]:
            self.table.setItem(0, 4, QTableWidgetItem(price_tov))
        else:
            self.table.setItem(0, 4, QTableWidgetItem(str(rozliv[0].get("price"))))
        #колво на складе
        if str(quantity[0].get('quantity'))=='-1.0':
            self.table.setItem(0, 5, QTableWidgetItem('---'))
        else:
            self.table.setItem(0, 5, QTableWidgetItem(str(quantity[0].get('quantity'))))
        #сумма
        if rozliv==[]:
            self.table.setItem(0, 6, QTableWidgetItem(price_tov))
        else:
            self.table.setItem(0, 6, QTableWidgetItem(str(rozliv[0].get("price"))))
            
        

        self.qle.clear()
        

        indices = int( self.table.rowCount())-1
        #self.save_izm(pole,quantity[0].get('id'),price[0].get('id'),self.qle1.text(),info_pokupka )

       # grid_layout = QGridLayout() 
        #grid_layout.addWidget(self.table, 0, 0)   # Добавляем таблицу в сетку
        id_pok=[]
        #запись в бд корзину
        rowPosition = self.table.rowCount()
 
        id_pok=[]
        #в таблице хранб id из бд
        connection.commit()
        cursor=connection.cursor()
        cursor.execute("SELECT id FROM kassa_basket where NUMBER_kassa="+str(a))
        for row1 in cursor: id_pok.append(row1)  
        self.table.setItem(0, 0, QTableWidgetItem(str(id_pok[len(id_pok)-1].get("id"))))
        self.table.item(0, 0).setBackground(QtGui.QColor(0,0,0))
        #делаю поле невозможным для просмотра
        self.table.item(0, 0).setFlags(QtCore.Qt.ItemIsEnabled) 
        self.table.item(0, 1).setFlags(QtCore.Qt.ItemIsEnabled)
        self.table.item(0, 3).setFlags(QtCore.Qt.ItemIsEnabled)
        self.table.item(0,5).setFlags(QtCore.Qt.ItemIsEnabled)
        self.table.item(0,6).setFlags(QtCore.Qt.ItemIsEnabled)
        
        
        self.table.item(0, 0).setBackground(QtGui.QColor(255, 255, 205))
        self.table.item(0, 0).setForeground(QBrush(QColor(255, 255, 205)))
        self.table.resizeRowsToContents () 
   
        
        self.table.cellChanged.connect(self.onCellChanged)
        
        btn1.setText("удалить")
        #функия в кнопку удаления
        btn1.clicked.connect(self.delete_stroka)

        self.summa()
        self.itogo()
    def zapis_BD(self,info_pokupka,quantity,price_id,qle1,cena):
        rowPosition = self.table.rowCount()
        a=self.label_number_chek.text()
        a=a[len(a)-1]
        
        cursor=connection.cursor()
        cursor.execute(
        r"""INSERT INTO kassa_basket (pos,kassa_goods_id,quantity,kassa_price_id,
        kassa_rests_id,kassa_ulonsaleplace_id,price,fullname,typ_id,
        edizm_id,shtrih,marka,alcnotsendedegais,markatovar,number_kassa) values("""
            +str(rowPosition)+","+str(info_pokupka[0].get('id'))+",'1',"+str(price_id)
            +','+str(quantity[0].get('id'))+","+str(quantity[0].get("kassa_ulonsaleplace_id"))+","
            +str(cena)+",'"+str(info_pokupka[0].get('fullname'))+"','3','"+str(info_pokupka[0].get('edizm_id'))+"','"+
            str(info_pokupka[0].get('shtrih'))+"','"
            +str(qle1)+"','1',' ',"+str(a)+")")
        
        connection.commit()
 
            
    def tovar_ots(self,w,h): 
        print("функция 1")
        self.lbl6.setText("Товар с таким штрих-кодом не обнаружен!")
        self.verticalLayout.insertWidget(6,self.lbl6)
        QTimer.singleShot(3000, self.udal_lbl)
        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
        
    def tovar_nalich(self,w,h): 
        print("функция 2")
        self.lbl6.setText("Товара нет в налчии!")
        self.verticalLayout.insertWidget(6,self.lbl6)
        QTimer.singleShot(3000, self.udal_lbl)
       
        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
                
    def tovar_net_cena(self,w,h):
        print("функция 3")
        
        self.lbl6.setText("Цена не назначена!")
        self.verticalLayout.insertWidget(6,self.lbl6)
        QTimer.singleShot(3000, self.udal_lbl)
        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
                
    def tovar_net_kolvo(self,w,h):
        print("функция 4")
        self.lbl6.setText("Количество не назначено!")
        self.verticalLayout.insertWidget(6,self.lbl6)
        QTimer.singleShot(3000, self.udal_lbl)
        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
                
    def tovar_alc_time(self):
        print("функция 4")
        self.lbl6.setText("Вы пытаетесь продать алкоголь в недоступное время!")
        self.verticalLayout.insertWidget(6,self.lbl6)
        QTimer.singleShot(3000, self.udal_lbl)
        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
        
        
    def tovar_never_vvod(self,w,h):

        self.lbl6.setText("Неверный ввод!")
        self.verticalLayout.insertWidget(6,self.lbl6)
        QTimer.singleShot(3000, self.udal_lbl)
        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)



        
    def udal_lbl(self):
        if self.lbl6 != None:
            self.verticalLayout.removeWidget(self.lbl6)
            sip.delete(self.lbl6)
            self.lbl6 = None
                
        
    #срабатываемые функции при нажатии на кнопки
    def keyReleaseEvent(self, e):
        rowPosition = self.table.rowCount()
        print(e.type())
        #нажимая на кнопку enter срабатывает функция штрих кода, если поле не пусто
        
        if (e.key()==16777220 or e.key()==16777221 ) and self.qle.hasFocus()==True and self.qle.text()!='':
            print("CHF<JNFKJ!")
            self.shtrih_kod()
    
        elif (e.key()==16777220 or e.key()==16777221 ) and self.qle1!=None and self.qle1.hasFocus()==True:
            #if len(self.qle1.text())==68:
            self.marka(self.qle1.text())
            
        elif e.type() == 7 and rowPosition>=0:
            print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            self.summa()
            self.itogo()
            
        '''

        #функция марки, если поле для марки не пусто и длинна поля 68 или 150 символов     
        elif (e.key()==16777220 ) and str(self.qle.text())!='' and len(self.qle1.text())==68:
            self.marka(self.qle1.text())
        elif (e.key()==16777220 ) and str(self.qle1.text())!='' and len(self.qle1.text())==150:
            self.marka(self.qle1.text())
        '''

    #функция маркировки
    def marka(self,qle1):
        self.lbl9=QLabel(self)
        self.lbl9.setStyleSheet("color:#FF0000;")
        self.lbl9.setFont(QtGui.QFont("Times", 25))
        
        
        print("ФУНКЦИЯ МАРКИ")

        desktop=QtWidgets.QApplication.desktop()
        w=desktop.width()/2
        h=desktop.height()
        #pole=str(self.qle.text())
        rowPosition = self.table.rowCount()
        layout = dict(zip(map(ord, "йцукенгшщзхъфывапролджэячсмитьбю.ё"
                           'ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ,Ё'),
                            "qwertyuiop[]asdfghjkl;'zxcvbnm,./`"
                           'QWERTYUIOP{}ASDFGHJKL:"ZXCVBNM<>?~'))
        qle1=qle1.translate(layout)
        
        quantity=[]
        if len(qle1)==68:
            fl=0
            kassa_good_id=[]
            sh=[]
            pole=qle1
            
            #марка старого образца достаём алкокод из неё
            alcocod='0'+self.convert_base(pole[7:19],to_base=10, from_base=36)
            print("alcocod",alcocod)
            #определяем есть ли запись с таким алкокодом в бд
            connection.commit()
            cursor=connection.cursor()
            cursor.execute("""
            SELECT kassa_goods_id 
            FROM  kassa_goods_alc 
            WHERE  AlcCode LIKE '"""+alcocod+"'ORDER BY  kassa_goods_alc. id ASC ")
            for row1 in cursor:kassa_good_id.append(row1)
            #определяем штрих код
            if len(kassa_good_id)==1:
                cursor.execute("SELECT shtrih FROM  kassa_goods WHERE id="+str(kassa_good_id[0].get("kassa_goods_id")))
                for row1 in cursor:sh.append(row1)
                print("штрих",sh)
                self.opredel_sh(sh,kassa_good_id[0].get("kassa_goods_id"),qle1,pole,fl)
                
            else: 
                self.otskan_marka()
                
        elif len(qle1)==150:
            print("1")
            alccod=[]
            kassa_good_id=[]
            data_egais=[]
            sh=[]   
            fl=1
            amc=[]
            pole=qle1
            #марка нового образца проверка на дату
            connection.commit()
            cursor2=connection.cursor()
            cursor2.execute("SELECT dt_sendtoegais FROM egais_restsshop_stockposition_amc where amc='"+qle1+"'")                                                                                                 
            for row1 in cursor2:data_egais.append(row1)
            if data_egais==[]:
                self.marka_ots()
                print("2")
                
                
            elif data_egais[0].get("dt_sendtoegais")!='0000-00-00 00:00:00':
                    self.marka_data()
                    #self.qle1.clear()
                    print("3")
            else:
                print("4")
                #алкокод
                connection.commit()
                cursor=connection.cursor()
                cursor.execute("SELECT alccode FROM egais_restsshop_stockposition_amc WHERE amc='"+qle1+"'") 
                for row1 in cursor:alccod.append(row1)
                #если алкокод не обнаружен
                if alccod==[]:
                    self.marka_ots()
                    print("5")
                else:
                    #опреляем штрих код
                    connection.commit()
                    cursor=connection.cursor()
                    print("6")
                    cursor.execute("""
                    SELECT kassa_goods_id 
                    FROM  kassa_goods_alc 
                    WHERE  AlcCode LIKE '"""+alccod[0].get("alccode")+"'ORDER BY  kassa_goods_alc. id ASC ")

                    for row1 in cursor:kassa_good_id.append(row1)
               
                    if len(kassa_good_id)==1:
                        print("7")
                        cursor.execute("SELECT shtrih FROM  kassa_goods WHERE id="+str(kassa_good_id[0].get("kassa_goods_id")))
                        for row1 in cursor:sh.append(row1)
                        self.opredel_sh(sh,kassa_good_id[0].get("kassa_goods_id"),qle1,pole,fl)

                    else: 
                        
                        self.otskan_marka()
        
        elif len(qle1)>20:
            sigaret=[]
            connection.commit()
            cursor2=connection.cursor()
            cursor2.execute("SELECT id,shtrih,markatovar FROM kassa_goods where shtrih='"+self.qle.text()+"'") 
            for row1 in cursor2:sigaret.append(row1)
            if sigaret==[] or sigaret[0].get("markatovar")==0:

                self.marka_nevern_vvod()
            else:
                self.opredel_sh(sigaret,sigaret[0].get("id"),qle1,qle1,3)
        else:
            self.marka_nevern_vvod()
            
                
                        

    def opredel_sh(self,sh,kassa_good_id,qle1,pole,fl):
        if len(qle1)!=68 and len(qle1)!=150 and fl!=3: 
            
            self.marka_ots()
        else:



            #если штрих код определён единственным образом
            if(len(sh)==1):
                pole=sh[0].get("shtrih")
                quantity=[]
                #берём данные о покупке
                print(kassa_good_id)
                connection.commit()
                cursor=connection.cursor()
                info_pokupka=[]
                cursor.execute("SELECT id,fullname,fullname2,alc,shtrih,edizm_id,marka,markatovar FROM kassa_goods where id="+str(kassa_good_id))
                for row1 in cursor: info_pokupka.append(row1)
                price=[]
                cursor1=connection.cursor()
                cursor1.execute("SELECT id,price FROM kassa_price where kassa_goods_id="+str(kassa_good_id))
                for row1 in cursor1: price.append(row1)

                cursor2=connection.cursor()
                cursor2.execute("SELECT id,quantity,kassa_ulonsaleplace_id FROM kassa_rests where kassa_goods_id="+str(info_pokupka[0].get('id')))
                for row1 in cursor2:quantity.append(row1)

                alc_cod=[]
                cursor2=connection.cursor()
                cursor2.execute("SELECT AlcCode FROM kassa_goods_alc where kassa_goods_id="+str(int(info_pokupka[0].get('id'))))
                for row1 in cursor2:alc_cod.append(row1)

                cursor2=connection.cursor()
                chek=[]
                cursor2.execute("SELECT marka FROM kassa_check_positions where marka='"+str(qle1)+"'")                                                                                                 
                for row1 in cursor2:chek.append(row1)

                cursor2=connection.cursor()
                korz=[]
                cursor2.execute("SELECT marka FROM kassa_basket where marka='"+str(qle1)+"'")                                                                                                   
                for row1 in cursor2:korz.append(row1)
                    
                times=[]
                cursor2=connection.cursor()   
                cursor2.execute("SELECT begintime,endtime FROM egais_config")
                for row1 in cursor2:times.append(row1)
                time = datetime.datetime.now().time()
                value=times[0].get("begintime")
                time_morning=(datetime.datetime.min + value).time()
                value=times[0].get("endtime")
                time_evn=(datetime.datetime.min + value).time()

                #развлетвление если марка старого и нового образца (скорее всего не нужно)
                if fl==3:
                    if info_pokupka==[] or int(info_pokupka[0].get("shtrih"))=='':
                        
                        self.shtrih_ne_naznach()
  
                    #цена неназначена
                    elif int(quantity[0].get("quantity"))<=0:
                        
                        self.marka_nalich()
                        
                        

                    #есть в таблице чеков
                    elif chek!=[]:
                        print("чеки",chek)
                        self.marka_data()

                    #есть в корзине
                    elif korz!=[]:
                        self.marka_korz()
                        
                    else:    
                        


                        if price!=[]:
                            self.new_stroka(info_pokupka,quantity,pole,str(price[0].get('price')),price[0].get("id"),qle1,price[0].get("price"),[])

                        else:
                            self.new_stroka(info_pokupka,quantity,pole,"0",'0',qle1,"0",[])


                        if self.qle1 != None:
                            self.verticalLayout_2.removeWidget(self.qle1)
                            sip.delete(self.qle1)
                            self.qle1 = None

                        if self.lbl9 != None:
                            self.verticalLayout_2.removeWidget(self.lbl9)
                            sip.delete(self.lbl9)
                            self.lbl9 = None

                        self.qle.clear()


                        self.qle.setDisabled(False)
                        self.qle.setFocus(0)                        
                    
                elif fl==1 or fl==0:

                    if info_pokupka==[] or int(info_pokupka[0].get("shtrih"))=='':
                        
                        self.shtrih_ne_naznach()
                    elif time<time_morning or time>time_evn :
                        self.tovar_alc_time()   
                    #цена неназначена
                    elif int(quantity[0].get("quantity"))<=0:
                        
                        self.marka_nalich()
                        
                        

                    #есть в таблице чеков
                    elif chek!=[]:
                        print("чеки",chek)
                        self.marka_data()

                    #есть в корзине
                    elif korz!=[]:
                        self.marka_korz()

                    #проверки пройдены, заполняем таблицу и записывааем в бд
                    else:
                        rozliv=[]
                        cursor1=connection.cursor()
                        for i in range(0,len(organizac)):
                            cursor1.execute("SELECT price, volume,edizmcode FROM egais_rozliv where FSRAR='"+
                            str( organizac[i].get('fsrar'))+"' AND ALCCODE='"+str(alc_cod[0].get('AlcCode'))+"'")
                            for row1 in cursor1: rozliv.append(row1)
                        if rozliv==[]:
                           
                            if self.qle1 != None:
                                self.verticalLayout_2.removeWidget(self.qle1)
                                sip.delete(self.qle1)
                                self.qle1 = None

                            if self.lbl9 != None:
                                self.verticalLayout_2.removeWidget(self.lbl9)
                                sip.delete(self.lbl9)
                                self.lbl9 = None

                            if price!=[]:
                                self.new_stroka(info_pokupka,quantity,pole,str(price[0].get('price')),price[0].get("id"),qle1,price[0].get("price"),[])
                            
                            else:
                                self.new_stroka(info_pokupka,quantity,pole,"0",'0',qle1,"0",[])
                                                            
                            self.qle.clear()


                            self.qle.setDisabled(False)
                            self.qle.setFocus(0)

                        else: 
                            
                            if self.qle1 != None:
                                self.verticalLayout_2.removeWidget(self.qle1)
                                sip.delete(self.qle1)
                                self.qle1 = None

                            if self.lbl9 != None:
                                self.verticalLayout_2.removeWidget(self.lbl9)
                                sip.delete(self.lbl9)
                                self.lbl9 = None

                            
                            #self.new_stroka(info_pokupka,quantity,pole,str(price[0].get('price')),price,qle1,price[0].get("price"),[])
                            self.qle.clear()


                            self.qle.setDisabled(False)
                            self.qle.setFocus(0)


                            self.new_stroka(info_pokupka,quantity,pole,str(price[0].get('price')),price[0].get("id"),qle1,price[0].get("price"),rozliv)
                                #self.rozliv_new_stroka(info_pokupka,rozliv)                        


            else:
                self.otskan_marka()
            
            
    def marka_ots(self):
        print("функция 6")
        if self.qle1 != None:
            self.verticalLayout.removeWidget(self.qle1)
            sip.delete(self.qle1)
            self.qle1 = None

        self.verticalLayout.insertWidget(6,self.lbl9)
        self.lbl9.setText("Марка не обнаружена!")
        QTimer.singleShot(3000, self.udal)


        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)

        
        
    def marka_data(self):
        print("функция 7")
  
        self.verticalLayout.removeWidget(self.qle1)
        if self.qle1 != None:
            self.verticalLayout.removeWidget(self.qle1)
            sip.delete(self.qle1)
            self.qle1 = None
            

                            

        self.verticalLayout.insertWidget(6,self.lbl9)
        self.lbl9.setText("Покупка была совершена!")
        QTimer.singleShot(3000, self.udal)


        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)

        
        
    def marka_korz(self):
        print("функция 8")
        self.verticalLayout.insertWidget(6,self.lbl9)
        self.lbl9.setText("Товар уже в коризне!!!")
        QTimer.singleShot(3000, self.udal)
        
        if self.qle1 != None:
            self.verticalLayout.removeWidget(self.qle1)
            sip.delete(self.qle1)
            self.qle1 = None
        

        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)  


        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
    def marka_cena(self):
        print("функция 9")
        self.verticalLayout.insertWidget(6,self.lbl9)
        self.lbl9.setText("Цена не назначена!")
        QTimer.singleShot(3000, self.udal)
        
        if self.qle1 != None:
            self.verticalLayout.removeWidget(self.qle1)
            sip.delete(self.qle1)
            self.qle1 = None

        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)        
    def marka_nalich(self):
        print("функция 10")
        self.verticalLayout_2.insertWidget(6,self.lbl9)
        self.lbl9.setText("Товара нет в наличии!")
        QTimer.singleShot(3000, self.udal)
        
        if self.qle1 != None:
            self.verticalLayout.removeWidget(self.qle1)
            sip.delete(self.qle1)
            self.qle1 = None

        



        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
        
    def otskan_marka(self):
        print("функция 11")
        self.verticalLayout.insertWidget(6,self.lbl9)
        self.lbl9.setText("Алкокод не найден!")
        QTimer.singleShot(3000, self.udal)
        if self.qle1 != None:
            self.verticalLayout.removeWidget(self.qle1)
            sip.delete(self.qle1)
            self.qle1 = None

        
        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
    def marka_nevern_vvod(self):
        print("функция 12")
        self.verticalLayout.insertWidget(6,self.lbl9)
        self.lbl9.setText("Неверный ввод!")
        QTimer.singleShot(3000, self.udal)
        if self.qle1 != None:
            self.verticalLayout.removeWidget(self.qle1)
            sip.delete(self.qle1)
            self.qle1 = None


        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
        
        #очистка полей при отмене воода   
        
    def shtrih_ne_naznach(self):
        print("функция 12")
        self.verticalLayout.insertWidget(6,self.lbl9)
        self.lbl9.setText("Штрих-код не назначен!")
        QTimer.singleShot(3000, self.udal)
        if self.qle1 != None:
            self.verticalLayout.removeWidget(self.qle1)
            sip.delete(self.qle1)
            self.qle1 = None


        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
                
    def udal(self):
        if self.lbl9 != None:
            self.verticalLayout.removeWidget(self.lbl9)
            sip.delete(self.lbl9)
            self.lbl9 = None
            
    def otmena_vvoda(self):
        self.verticalLayout.insertWidget(6,self.lbl9)
        self.lbl9.setText("Марка не обнаружена!")
        QTimer.singleShot(3000, self.lbl9.clear)
        if self.qle1 != None:
            self.verticalLayout.removeWidget(self.qle1)
            sip.delete(self.qle1)
            self.qle1 = None

        self.qle.clear()
        self.qle.setDisabled(False)
        self.qle.setFocus(0)
    '''
    def keyPressEvent(self, e):
        if e.type() == 6:
            start_time = time.time()
            print("--- %s seconds ---" % (time.time() - start_time))

            
            
            #self.summa()

    def mousePressEvent(self, e):
        if e.type() == 2:
            pass
            #self.summa()
     
    '''    



            
        
    #удаление строки     
    def delete_stroka(self): 
        id_pok=[]
        indices = self.table.selectionModel().currentIndex().row()
        if indices!=-1:
            
            print("Номер строки",indices)
            #удаление из корзины по id 
            cursor=connection.cursor()
            cursor.execute("""
            DELETE FROM kassa_basket WHERE  id="""+str(int(self.table.item(indices ,0).text())))
            connection.commit()
            self.table.removeRow(indices)
            self.qle.setFocus(0)
            #пересёт итьоговой суммы
            self.itogo()

    #поиск изменений в таблицу
    def onCellChanged(self, row, column):
        #выделенная строка
        indices = self.table.selectionModel().currentIndex().row()

        
        if indices!=-1 and self.table.item(indices,2)!=None:
            item = self.table.item(row, column)
            lastState = item.data(LastStateRole)
            currentState = item.checkState()
            kassa_price_id=[]
            #если тпблица изменилась
            if currentState != lastState and self.table.item(indices,2).text()!='' and self.table.item(indices,4).text()!='':
                #обновляется колво и цена
                
                rowPosition = self.table.rowCount()
                cursor=connection.cursor()
                cursor.execute(r"UPDATE  kassa_basket SET quantity="+self.table.item(indices,2).text()+
                " WHERE id="+self.table.item(indices,0).text())
                connection.commit()
                cursor.execute(r"UPDATE  kassa_basket SET price="+self.table.item(indices,4).text()+
                " WHERE id="+self.table.item(indices,0).text())
                connection.commit()
                cursor.execute(r"SELECT  kassa_price_id FROM KASSA_BASKET WHERE id="+self.table.item(indices,0).text())
                for row1 in cursor: kassa_price_id.append(row1)
                cursor.execute(r"UPDATE  kassa_price SET price="+self.table.item(indices,4).text()+
                " WHERE id="+str(kassa_price_id[0].get("kassa_price_id")))
                
                connection.commit()
                
                
              
                
  

    def on_combobox_changed(self,rozliv,combo_v):
        indices = self.table.selectionModel().currentIndex().row()
        #print("идекс",self.table.item(indices,2))
        for i in rozliv:
            if str(i.get("volume"))==combo_v.currentText():
                price=i.get("price")
                self.table.setItem(indices, 4, QTableWidgetItem(str(price)))
                self.table.setItem(indices, 6, QTableWidgetItem(str(price)))
            else:
                
                price=rozliv[0].get("price")                
        

        
    '''
    def onChanged(self):
        self.keyReleaseEvent()
    '''
    #функция, срабатывающая при не пустой корзине при открытии приложения
    def zapolnenie(self,number_shek):
        
        desktop=QtWidgets.QApplication.desktop()
        w=desktop.width()/2
        h=desktop.height()


        self.table.setHorizontalHeaderLabels(["","НАИМЕНОВАНИЕ", "КОЛ-ВО","-","   ЦЕНА   "," ОСТАТОК ","   СУММА   "," "])
        fnt = self.table.font()
        fnt.setPointSize(30)

        for i in range(0,8):
            self.table.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)

        self.table.setSelectionBehavior(QAbstractItemView.SelectRows)
        #колонки по размерности текста
        header = self.table.horizontalHeader()       
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)
        #колонка названия товара растягивается максимально
        header.setSectionResizeMode(1, QtWidgets.QHeaderView.Stretch)
        for i in range(2,8):
            header.setSectionResizeMode(i, QtWidgets.QHeaderView.ResizeToContents)

        indices = int(self.table.selectionModel().currentIndex().row())

        
            
        info_pokupka=[]
        connection.commit()
        cursor1=connection.cursor()
        
        cursor1.execute("SELECT kassa_goods_id,pos,quantity,fullname, edizm_id,price,kassa_rests_id,shtrih,marka FROM kassa_basket WHERE NUMBER_kassa="+str(number_shek))
        for row1 in cursor1: info_pokupka.append(row1)
        info_pokupka.reverse()
        

        id_pok=[]
        cursor=connection.cursor()
        cursor.execute("SELECT id FROM kassa_basket where NUMBER_kassa="+str(number_shek))
        for row1 in cursor: id_pok.append(row1)
        id_pok.reverse()

        
        quantity=[]
        ed_izm=[]
        for i in range (0,len(info_pokupka)):

            cursor1=connection.cursor()
            cursor1.execute("SELECT quantity FROM kassa_rests where kassa_goods_id="+str(info_pokupka[i].get('kassa_goods_id')))
            for row1 in cursor1:quantity.append(row1)
        #quantity.reverse()

        for i in range (0,len(info_pokupka)):
            if info_pokupka[i].get('edizm_id')=="ed":
                ed_izm.append("шт    ")
                
            if info_pokupka[i].get('edizm_id')=="l":
                ed_izm.append("л    ")
            if info_pokupka[i].get('edizm_id')=="kg":
                ed_izm.append("кг    ")
                
        #self.table.doubleClicked.connect(self.click_clear)
        for i in range (0,len(info_pokupka)):
            self.table.insertRow(i) 
            btnkn = QPushButton()
            
            btnkn.setText('удалить')


            btnkn.clicked.connect(self.delete_stroka)
            self.table.setRowHeight(i,60) 
                
            
            #заполняем все стобцы
            self.table.setCellWidget(i, 7, btnkn)         
            #self.table.setItem(i, 0, QTableWidgetItem("№"+str(info_pokupka[i].get('pos'))))
            self.table.setItem(i, 1, QTableWidgetItem(info_pokupka[i].get('fullname')+'\n'+info_pokupka[i].get('shtrih')+'\n'+info_pokupka[i].get('marka')))
            
            self.table.setItem(i, 2, QTableWidgetItem(str(info_pokupka[i].get('quantity'))))
            self.table.setItem(i, 3, QTableWidgetItem(ed_izm[i]))
            self.table.setItem(i, 4, QTableWidgetItem(str(info_pokupka[i].get('price'))))
            if str(quantity[i].get('quantity'))=='-1.0':
                self.table.setItem(i, 5, QTableWidgetItem('---'))
            else:
                self.table.setItem(i, 5, QTableWidgetItem(str(quantity[i].get('quantity'))))
                
            self.table.setItem(i, 6, QTableWidgetItem(str(float(info_pokupka[i].get('quantity'))*
                                                          float(info_pokupka[i].get('price')))))
            self.table.setItem(i, 0, QTableWidgetItem(str(id_pok[i].get("id"))))
            
            
            self.table.item(i, 0).setFlags(QtCore.Qt.ItemIsEnabled)
            self.table.item(i, 1).setFlags(QtCore.Qt.ItemIsEnabled)
            if info_pokupka[i].get("marka")!=' ':
                self.table.item(i,2).setFlags(QtCore.Qt.ItemIsEnabled)
                
            self.table.item(i,3).setFlags(QtCore.Qt.ItemIsEnabled)    
            self.table.item(i,5).setFlags(QtCore.Qt.ItemIsEnabled)
            self.table.item(i, 0).setBackground(QtGui.QColor(255, 255, 205))
            self.table.item(i, 0).setForeground(QBrush(QColor(255, 255, 205)))
        self.table.cellChanged.connect(self.onCellChanged)
        self.table.resizeRowsToContents () 

        self.itogo()

        
    def click_clear(self):
        indices1 = int(self.table.selectionModel().currentIndex().row())
        indices2 = int(self.table.selectionModel().currentIndex().column())
        print(indices1,indices2)
        if indices2==2 or indices2==4:
            self.table.setItem(indices1, indices2, QTableWidgetItem(''))
    #перевод из 36 системы в 10 (марка старого образца)
    def convert_base(self,num, to_base=10, from_base=36):
        to_base=10
        from_base=36
        if isinstance(num, str):
            n = int(num, from_base)
        else:
            n = int(num)
        alphabet = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        if n < to_base:
            return alphabet[n]
        else:
            return self.convert_base(n // to_base, to_base) + alphabet[n % to_base] 
    #открыть окно для ввода нового товара
    def open_dialog (self,info_pokupka ) :
        dialog = dop_okno(self,info_pokupka)
        dialog .setWindowModality(Qt.ApplicationModal)
        result = dialog.exec_()
        if result == QtWidgets.QDialog.Accepted:
            self.qle.setFocus(0)

            info_pokupka1=[]
            i=dialog.table1.selectionModel().currentIndex().row()
            print(info_pokupka[i])
            info_pokupka1.append(info_pokupka[i])
            
            self.alone_shtrich(info_pokupka1)
            
        else:
            self.qle.clear()
        
                
    def open_dialog2 (self) :
        dialog = dop_okno_podbor(self)
        result = dialog.exec_()
        
        #self.dialog.setWindowModality(Qt.ApplicationModal)
        if result == QtWidgets.QDialog.Accepted:
            self.qle.setFocus(0)
            i=dialog.table1.selectionModel().currentIndex().row()
            a=dialog.table1.item(i,0).text()
            connection.commit()
            info_pokupka=[]
            cursor2=connection.cursor()
            cursor2.execute("SELECT id,fullname,shtrih,edizm_id,fullname2,marka,markatovar FROM kassa_goods where id="+str(a))
            for row1 in cursor2: info_pokupka.append(row1)
            print(info_pokupka)
            print(a)
            self.alone_shtrich(info_pokupka)        


    def open_dialog_3 (self ) :
        dialog = Vvod_new_tovar(self)
        dialog.setWindowModality(Qt.ApplicationModal)
        result = dialog.exec_()
        

        # Здесь получаем данные из диалогового окна
        if result == QtWidgets.QDialog.Accepted:
            self.qle.setFocus(0)
            print("Нажата кнопка Cancel")
            connection.commit()
            cursor2=connection.cursor()
            cursor2.execute("SELECT MAX(id) FROM kassa_GOODS ")
            number_id=[]
            for row1 in cursor2: number_id.append(row1) 
            info_pokupka1=[]
            cursor2=connection.cursor()    
            cursor2.execute("SELECT id,fullname,shtrih,edizm_id,fullname2,marka,markatovar FROM kassa_goods where id="+str(number_id[0].get('MAX(id)')))
            for row1 in cursor2: info_pokupka1.append(row1) 
                    

            self.alone_shtrich(info_pokupka1)  

            
    def open_dialog_4 (self ) :
        dialog = dop_okno_korz(self)
        dialog.setWindowModality(Qt.ApplicationModal)
        result = dialog.exec_()
        kassa=[]
        connection.commit()
        for i in range(0,len(ul)):
            cursor2=connection.cursor()
            cursor2.execute("SELECT price,number_kassa FROM kassa_basket")
        for row1 in cursor2: kassa.append(row1)
        a=0
        
        
        kassa_number=[]
        for i in kassa:
            kassa_number.append(i.get("number_kassa"))
            
            
        c = Counter(kassa_number)
        
        c=list(c)
        
        
        # Здесь получаем данные из диалогового окна
        if result == QtWidgets.QDialog.Accepted:
            self.qle.setFocus(0)
            i=dialog.table1.selectionModel().currentIndex().row()
            print("ИДЕКС СТРОКИ",i,kassa_number)
            self.label_number_chek.setText("Чек №"+str(c[i]))

            position=[]
           
            
            connection.commit()
            cursor1=connection.cursor()
            cursor1.execute("SELECT COUNT(*) FROM kassa_basket WHERE NUMBER_KASSA="+str(c[i]))
            for row1 in cursor1: position.append(row1)
            if position[0].get('COUNT(*)')!=0:
                    self.table.setRowCount(0)
                    self.zapolnenie(c[i])
                    self.table.cellChanged.connect(self.onCellChanged)                

            else: 


                self.table.setRowCount(0)
                self.table.cellChanged.connect(self.onCellChanged) 
                

    def open_dialog_5 (self ) :
        dialog = dop_okno_opl(self)
        dialog.setWindowModality(Qt.ApplicationModal)
        result = dialog.exec_()                

        

        
        
    def open_dialog_6 (self):
        if not self.gurnal_prWin:
            self.gurnal_prWin = gurnal_pr()
        self.gurnal_prWin.show()
        
    def quitApp(self): 
        QtCore.QCoreApplication.instance().quit() 
        
        
    
    def closeEvent(self, e):
        
        result = QtWidgets.QMessageBox.question(self,
        "Подтверждение закрытия окна",
        "Вы действительно хотите закрыть ОБЛАКО?",
         QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No,
         QtWidgets.QMessageBox.No)
        if result == QtWidgets.QMessageBox.Yes:
            e.accept()
            QtWidgets.QWidget.closeEvent(self, e)
        else:
            e.ignore()
class Controller:

    def __init__(self):
        pass

    def open_win1(self):
        self.window_two = Example()
        self.window_two.show() 
        #if not self.vvodnewtovarWin:
        #self.vvodnewtovarWin = Vvod_new_tovar(self)
        #self.vvodnewtovarWin.setWindowModality(Qt.ApplicationModal)

def main():
    #appctxt = ApplicationContext()       # 1. Instantiate ApplicationContext
    if QApplication.instance() != None: 
        app = QApplication.instance() 
    else: 
        app = QApplication(sys.argv) 
    controller = Controller()
    controller.open_win1()
    sys.exit(app.exec_())
    #exit_code = appctxt.app.exec_()      # 2. Invoke appctxt.app.exec_()
    #sys.exit(exit_code)    
if __name__ == '__main__':  # Если мы з файл напрямую, а не импортируем
    main()  # то запускаем функцию main()

1 фукнция
СРАБОТАЛО
1 фукнция
СРАБОТАЛО
1 фукнция
СРАБОТАЛО
1 фукнция
СРАБОТАЛО
1 фукнция
СРАБОТАЛО
1 фукнция
СРАБОТАЛО
1 фукнция
СРАБОТАЛО
1 фукнция
СРАБОТАЛО
1 фукнция
СРАБОТАЛО
1 фукнция
СРАБОТАЛО
[{'id': 1451, 'fullname': 'ррр', 'shtrih': '545354', 'edizm_id': 'ed', 'fullname2': '', 'marka': 0, 'markatovar': 0}]
1451
ФУНКЦИЯ ЕДНСТВЕННЫЙ ШТРИХ КОД
номер кассы 1
[{'id': 1448, 'fullname': 'ТВОРОГ', 'shtrih': '83212', 'edizm_id': 'ed', 'fullname2': '', 'marka': 0, 'markatovar': 0}]
1448
ФУНКЦИЯ ЕДНСТВЕННЫЙ ШТРИХ КОД
номер кассы 1
[{'id': 1453, 'fullname': 'творог 2', 'shtrih': '32213', 'edizm_id': 'ed', 'fullname2': '', 'marka': 0, 'markatovar': 0}]
1453
ФУНКЦИЯ ЕДНСТВЕННЫЙ ШТРИХ КОД
номер кассы 1
[{'id': 1453, 'fullname': 'творог 2', 'shtrih': '32213', 'edizm_id': 'ed', 'fullname2': '', 'marka': 0, 'markatovar': 0}]
1453
ФУНКЦИЯ ЕДНСТВЕННЫЙ ШТРИХ КОД
номер кассы 1
Номер строки 3
Номер строки 3
Номер строки 3
Номер строки 3
Номер строки 3
Номер строки 3
Номер строки 3
Номер строки 3
Но

SystemExit: 0

In [20]:
datetime.datetime.now().date()

datetime.date(2019, 8, 16)